In [26]:
import pandas as pd
import numpy as np


In [27]:
dataAll = pd.read_csv('/Users/ruben.arana/git/beat-bookies/data/combinedData.csv', low_memory=False)

## CREATE TEAM EMBEDDINGS

In [28]:
# Create indexes for teams
teamId2idx = {o:i for i,o in enumerate(dataAll['VISITOR_TEAM'])}
teamId2idx

{'Atlanta': 1174,
 'Boston': 1192,
 'Brooklyn': 1188,
 'Charlotte': 1146,
 'Chicago': 1155,
 'Cleveland': 1197,
 'Dallas': 1172,
 'Denver': 1193,
 'Detroit': 1183,
 'Golden State': 1156,
 'Houston': 1189,
 'Indiana': 1191,
 'L.A. Clippers': 1200,
 'L.A. Lakers': 1144,
 'Memphis': 1185,
 'Miami': 1190,
 'Milwaukee': 1186,
 'Minnesota': 1181,
 'New Orleans': 1179,
 'New York': 1199,
 'Orlando': 1173,
 'Philadelphia': 1198,
 'Phoenix': 1184,
 'Portland': 1162,
 'Sacramento': 1137,
 'San Antonio': 1164,
 'Toronto': 1196,
 'Utah': 1195,
 'Washington': 1169}

In [29]:
# Replace all team nambes for the indexes
dataAll.VISITOR_TEAM = dataAll.VISITOR_TEAM.apply(lambda x: teamId2idx[x])
dataAll.HOME_TEAM = dataAll.HOME_TEAM.apply(lambda x: teamId2idx[x])

## GET AVERAGE STATS GIVEN A TEAM PREVIOUS TO A ROW

In [30]:
# Return all rows where where a specific team has played
def getTeamStats(dataframe, team, homeOrAway=None):
    if (homeOrAway is None):
        return dataframe.loc[(dataframe["VISITOR_TEAM"]== team) | (dataframe["HOME_TEAM"]== team)]
    if (homeOrAway == "HOME"):
        return dataframe.loc[(dataframe["HOME_TEAM"]== team)]
    if (homeOrAway == "AWAY") :
        return dataframe.loc[(dataframe["VISITOR_TEAM"]== team)]

In [31]:
# Return all rows before an specific date
def getRowsBefore(dataframe, year, month, day):
    return dataframe.loc[
        (dataframe["YEAR"]< year) 
        | ((dataframe["YEAR"]== year) & (dataframe["MONTH"]<= month))
        | ((dataframe["YEAR"]== year) & (dataframe["MONTH"]== month) & (dataframe["DAY"] < day))
    ]

##  GENEARTING A NEW DATASET WITH COMPUTED AVERAGES

In [32]:
new_rows = []
for row in dataAll.itertuples():
    
    # Only HOME games for HOME team, only VISITOR games for VISITOR team DF
    visitorTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).mean()
    homeTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).mean()

    # Only HOME games for VISITOR team, only VISITOR games for HOME teams DF
    visitorTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).mean()
    homeTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).mean()

    # Only last 6 games (HOME games for HOME team, only VISITOR games for VISITOR) team DF
    last6visitorTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).tail(6).mean()
    last6homeTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).tail(6).mean()

    # Only last 6 games (HOME games for VISITOR team, only VISITOR games for HOME team) DF
    last6visitorTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).tail(6).mean()
    last6homeTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).tail(6).mean()

    # Only last 3 games (HOME games for HOME team, only VISITOR games for VISITOR) team DF
    last3visitorTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).tail(3).mean()
    last3homeTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).tail(3).mean()

    # Only last 3 games (HOME games for VISITOR team, only VISITOR games for HOME team) DF
    last3visitorTeamAsHomeDf = getRowsBefore(getTeamStats(dataAll, row.VISITOR_TEAM, "HOME"), row.YEAR, row.MONTH, row.DAY).tail(3).mean()
    last3homeTeamAsVisitorDf = getRowsBefore(getTeamStats(dataAll, row.HOME_TEAM, "AWAY"), row.YEAR, row.MONTH, row.DAY).tail(3).mean()


    new_rows.append(
        [
            row.VISITOR_TEAM,
            row.HOME_TEAM, 
            
            #Totals visitor as visitor

            visitorTeamAsVisitorDf['VISITOR_FG'], 
            visitorTeamAsVisitorDf['VISITOR_FGA'],
            visitorTeamAsVisitorDf['VISITOR_FG%'],
            visitorTeamAsVisitorDf['VISITOR_3P'],
            visitorTeamAsVisitorDf['VISITOR_3PA'],
            visitorTeamAsVisitorDf['VISITOR_3P%'],
            visitorTeamAsVisitorDf['VISITOR_FT'],
            visitorTeamAsVisitorDf['VISITOR_FTA'],
            visitorTeamAsVisitorDf['VISITOR_FT%'],
            visitorTeamAsVisitorDf['VISITOR_ORB'],
            visitorTeamAsVisitorDf['VISITOR_DRB'],
            visitorTeamAsVisitorDf['VISITOR_TRB'],
            visitorTeamAsVisitorDf['VISITOR_AST'],
            visitorTeamAsVisitorDf['VISITOR_STL'],
            visitorTeamAsVisitorDf['VISITOR_BLK'],
            visitorTeamAsVisitorDf['VISITOR_TOV'],
            visitorTeamAsVisitorDf['VISITOR_PF'],
            visitorTeamAsVisitorDf['VISITOR_PTS'],
            visitorTeamAsVisitorDf['VISITOR_TS%'],
            visitorTeamAsVisitorDf['VISITOR_eFG%'],
            visitorTeamAsVisitorDf['VISITOR_3PAr'],
            visitorTeamAsVisitorDf['VISITOR_FTr'],
            visitorTeamAsVisitorDf['VISITOR_ORB%'],
            visitorTeamAsVisitorDf['VISITOR_DRB%'],
            visitorTeamAsVisitorDf['VISITOR_TRB%'],
            visitorTeamAsVisitorDf['VISITOR_AST%'], 
            visitorTeamAsVisitorDf['VISITOR_STL%'], 
            visitorTeamAsVisitorDf['VISITOR_BLK%'],
            visitorTeamAsVisitorDf['VISITOR_TOV%'],
            visitorTeamAsVisitorDf['VISITOR_USG%'],
            visitorTeamAsVisitorDf['VISITOR_ORtg'],
            visitorTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against visitor as visitor

            visitorTeamAsVisitorDf['HOME_FG'], 
            visitorTeamAsVisitorDf['HOME_FGA'],
            visitorTeamAsVisitorDf['HOME_FG%'],
            visitorTeamAsVisitorDf['HOME_3P'],
            visitorTeamAsVisitorDf['HOME_3PA'],
            visitorTeamAsVisitorDf['HOME_3P%'],
            visitorTeamAsVisitorDf['HOME_FT'],
            visitorTeamAsVisitorDf['HOME_FTA'],
            visitorTeamAsVisitorDf['HOME_FT%'],
            visitorTeamAsVisitorDf['HOME_ORB'],
            visitorTeamAsVisitorDf['HOME_DRB'],
            visitorTeamAsVisitorDf['HOME_TRB'],
            visitorTeamAsVisitorDf['HOME_AST'],
            visitorTeamAsVisitorDf['HOME_STL'],
            visitorTeamAsVisitorDf['HOME_BLK'],
            visitorTeamAsVisitorDf['HOME_TOV'],
            visitorTeamAsVisitorDf['HOME_PF'],
            visitorTeamAsVisitorDf['HOME_PTS'],
            visitorTeamAsVisitorDf['HOME_TS%'],
            visitorTeamAsVisitorDf['HOME_eFG%'],
            visitorTeamAsVisitorDf['HOME_3PAr'],
            visitorTeamAsVisitorDf['HOME_FTr'],
            visitorTeamAsVisitorDf['HOME_ORB%'],
            visitorTeamAsVisitorDf['HOME_DRB%'],
            visitorTeamAsVisitorDf['HOME_TRB%'],
            visitorTeamAsVisitorDf['HOME_AST%'], 
            visitorTeamAsVisitorDf['HOME_STL%'], 
            visitorTeamAsVisitorDf['HOME_BLK%'],
            visitorTeamAsVisitorDf['HOME_TOV%'],
            visitorTeamAsVisitorDf['HOME_USG%'],
            visitorTeamAsVisitorDf['HOME_ORtg'],
            visitorTeamAsVisitorDf['HOME_DRtg'],


            #Totals visitor as home

            visitorTeamAsHomeDf['HOME_FG'], 
            visitorTeamAsHomeDf['HOME_FGA'],
            visitorTeamAsHomeDf['HOME_FG%'],
            visitorTeamAsHomeDf['HOME_3P'],
            visitorTeamAsHomeDf['HOME_3PA'],
            visitorTeamAsHomeDf['HOME_3P%'],
            visitorTeamAsHomeDf['HOME_FT'],
            visitorTeamAsHomeDf['HOME_FTA'],
            visitorTeamAsHomeDf['HOME_FT%'],
            visitorTeamAsHomeDf['HOME_ORB'],
            visitorTeamAsHomeDf['HOME_DRB'],
            visitorTeamAsHomeDf['HOME_TRB'],
            visitorTeamAsHomeDf['HOME_AST'],
            visitorTeamAsHomeDf['HOME_STL'],
            visitorTeamAsHomeDf['HOME_BLK'],
            visitorTeamAsHomeDf['HOME_TOV'],
            visitorTeamAsHomeDf['HOME_PF'],
            visitorTeamAsHomeDf['HOME_PTS'],
            visitorTeamAsHomeDf['HOME_TS%'],
            visitorTeamAsHomeDf['HOME_eFG%'],
            visitorTeamAsHomeDf['HOME_3PAr'],
            visitorTeamAsHomeDf['HOME_FTr'],
            visitorTeamAsHomeDf['HOME_ORB%'],
            visitorTeamAsHomeDf['HOME_DRB%'],
            visitorTeamAsHomeDf['HOME_TRB%'],
            visitorTeamAsHomeDf['HOME_AST%'], 
            visitorTeamAsHomeDf['HOME_STL%'], 
            visitorTeamAsHomeDf['HOME_BLK%'],
            visitorTeamAsHomeDf['HOME_TOV%'],
            visitorTeamAsHomeDf['HOME_USG%'],
            visitorTeamAsHomeDf['HOME_ORtg'],
            visitorTeamAsHomeDf['HOME_DRtg'],

            #Totals against visitor as home

            visitorTeamAsHomeDf['VISITOR_FG'], 
            visitorTeamAsHomeDf['VISITOR_FGA'],
            visitorTeamAsHomeDf['VISITOR_FG%'],
            visitorTeamAsHomeDf['VISITOR_3P'],
            visitorTeamAsHomeDf['VISITOR_3PA'],
            visitorTeamAsHomeDf['VISITOR_3P%'],
            visitorTeamAsHomeDf['VISITOR_FT'],
            visitorTeamAsHomeDf['VISITOR_FTA'],
            visitorTeamAsHomeDf['VISITOR_FT%'],
            visitorTeamAsHomeDf['VISITOR_ORB'],
            visitorTeamAsHomeDf['VISITOR_DRB'],
            visitorTeamAsHomeDf['VISITOR_TRB'],
            visitorTeamAsHomeDf['VISITOR_AST'],
            visitorTeamAsHomeDf['VISITOR_STL'],
            visitorTeamAsHomeDf['VISITOR_BLK'],
            visitorTeamAsHomeDf['VISITOR_TOV'],
            visitorTeamAsHomeDf['VISITOR_PF'],
            visitorTeamAsHomeDf['VISITOR_PTS'],
            visitorTeamAsHomeDf['VISITOR_TS%'],
            visitorTeamAsHomeDf['VISITOR_eFG%'],
            visitorTeamAsHomeDf['VISITOR_3PAr'],
            visitorTeamAsHomeDf['VISITOR_FTr'],
            visitorTeamAsHomeDf['VISITOR_ORB%'],
            visitorTeamAsHomeDf['VISITOR_DRB%'],
            visitorTeamAsHomeDf['VISITOR_TRB%'],
            visitorTeamAsHomeDf['VISITOR_AST%'], 
            visitorTeamAsHomeDf['VISITOR_STL%'], 
            visitorTeamAsHomeDf['VISITOR_BLK%'],
            visitorTeamAsHomeDf['VISITOR_TOV%'],
            visitorTeamAsHomeDf['VISITOR_USG%'],
            visitorTeamAsHomeDf['VISITOR_ORtg'],
            visitorTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as home

            homeTeamAsHomeDf['HOME_FG'], 
            homeTeamAsHomeDf['HOME_FGA'],
            homeTeamAsHomeDf['HOME_FG%'],
            homeTeamAsHomeDf['HOME_3P'],
            homeTeamAsHomeDf['HOME_3PA'],
            homeTeamAsHomeDf['HOME_3P%'],
            homeTeamAsHomeDf['HOME_FT'],
            homeTeamAsHomeDf['HOME_FTA'],
            homeTeamAsHomeDf['HOME_FT%'],
            homeTeamAsHomeDf['HOME_ORB'],
            homeTeamAsHomeDf['HOME_DRB'],
            homeTeamAsHomeDf['HOME_TRB'],
            homeTeamAsHomeDf['HOME_AST'],
            homeTeamAsHomeDf['HOME_STL'],
            homeTeamAsHomeDf['HOME_BLK'],
            homeTeamAsHomeDf['HOME_TOV'],
            homeTeamAsHomeDf['HOME_PF'],
            homeTeamAsHomeDf['HOME_PTS'],
            homeTeamAsHomeDf['HOME_TS%'],
            homeTeamAsHomeDf['HOME_eFG%'],
            homeTeamAsHomeDf['HOME_3PAr'],
            homeTeamAsHomeDf['HOME_FTr'],
            homeTeamAsHomeDf['HOME_ORB%'],
            homeTeamAsHomeDf['HOME_DRB%'],
            homeTeamAsHomeDf['HOME_TRB%'],
            homeTeamAsHomeDf['HOME_AST%'], 
            homeTeamAsHomeDf['HOME_STL%'], 
            homeTeamAsHomeDf['HOME_BLK%'],
            homeTeamAsHomeDf['HOME_TOV%'],
            homeTeamAsHomeDf['HOME_USG%'],
            homeTeamAsHomeDf['HOME_ORtg'],
            homeTeamAsHomeDf['HOME_DRtg'],


            #Totals against home as home

            homeTeamAsHomeDf['VISITOR_FG'], 
            homeTeamAsHomeDf['VISITOR_FGA'],
            homeTeamAsHomeDf['VISITOR_FG%'],
            homeTeamAsHomeDf['VISITOR_3P'],
            homeTeamAsHomeDf['VISITOR_3PA'],
            homeTeamAsHomeDf['VISITOR_3P%'],
            homeTeamAsHomeDf['VISITOR_FT'],
            homeTeamAsHomeDf['VISITOR_FTA'],
            homeTeamAsHomeDf['VISITOR_FT%'],
            homeTeamAsHomeDf['VISITOR_ORB'],
            homeTeamAsHomeDf['VISITOR_DRB'],
            homeTeamAsHomeDf['VISITOR_TRB'],
            homeTeamAsHomeDf['VISITOR_AST'],
            homeTeamAsHomeDf['VISITOR_STL'],
            homeTeamAsHomeDf['VISITOR_BLK'],
            homeTeamAsHomeDf['VISITOR_TOV'],
            homeTeamAsHomeDf['VISITOR_PF'],
            homeTeamAsHomeDf['VISITOR_PTS'],
            homeTeamAsHomeDf['VISITOR_TS%'],
            homeTeamAsHomeDf['VISITOR_eFG%'],
            homeTeamAsHomeDf['VISITOR_3PAr'],
            homeTeamAsHomeDf['VISITOR_FTr'],
            homeTeamAsHomeDf['VISITOR_ORB%'],
            homeTeamAsHomeDf['VISITOR_DRB%'],
            homeTeamAsHomeDf['VISITOR_TRB%'],
            homeTeamAsHomeDf['VISITOR_AST%'], 
            homeTeamAsHomeDf['VISITOR_STL%'], 
            homeTeamAsHomeDf['VISITOR_BLK%'],
            homeTeamAsHomeDf['VISITOR_TOV%'],
            homeTeamAsHomeDf['VISITOR_USG%'],
            homeTeamAsHomeDf['VISITOR_ORtg'],
            homeTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as visitor

            homeTeamAsVisitorDf['VISITOR_FG'], 
            homeTeamAsVisitorDf['VISITOR_FGA'],
            homeTeamAsVisitorDf['VISITOR_FG%'],
            homeTeamAsVisitorDf['VISITOR_3P'],
            homeTeamAsVisitorDf['VISITOR_3PA'],
            homeTeamAsVisitorDf['VISITOR_3P%'],
            homeTeamAsVisitorDf['VISITOR_FT'],
            homeTeamAsVisitorDf['VISITOR_FTA'],
            homeTeamAsVisitorDf['VISITOR_FT%'],
            homeTeamAsVisitorDf['VISITOR_ORB'],
            homeTeamAsVisitorDf['VISITOR_DRB'],
            homeTeamAsVisitorDf['VISITOR_TRB'],
            homeTeamAsVisitorDf['VISITOR_AST'],
            homeTeamAsVisitorDf['VISITOR_STL'],
            homeTeamAsVisitorDf['VISITOR_BLK'],
            homeTeamAsVisitorDf['VISITOR_TOV'],
            homeTeamAsVisitorDf['VISITOR_PF'],
            homeTeamAsVisitorDf['VISITOR_PTS'],
            homeTeamAsVisitorDf['VISITOR_TS%'],
            homeTeamAsVisitorDf['VISITOR_eFG%'],
            homeTeamAsVisitorDf['VISITOR_3PAr'],
            homeTeamAsVisitorDf['VISITOR_FTr'],
            homeTeamAsVisitorDf['VISITOR_ORB%'],
            homeTeamAsVisitorDf['VISITOR_DRB%'],
            homeTeamAsVisitorDf['VISITOR_TRB%'],
            homeTeamAsVisitorDf['VISITOR_AST%'], 
            homeTeamAsVisitorDf['VISITOR_STL%'], 
            homeTeamAsVisitorDf['VISITOR_BLK%'],
            homeTeamAsVisitorDf['VISITOR_TOV%'],
            homeTeamAsVisitorDf['VISITOR_USG%'],
            homeTeamAsVisitorDf['VISITOR_ORtg'],
            homeTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against home as visitor

            homeTeamAsVisitorDf['HOME_FG'], 
            homeTeamAsVisitorDf['HOME_FGA'],
            homeTeamAsVisitorDf['HOME_FG%'],
            homeTeamAsVisitorDf['HOME_3P'],
            homeTeamAsVisitorDf['HOME_3PA'],
            homeTeamAsVisitorDf['HOME_3P%'],
            homeTeamAsVisitorDf['HOME_FT'],
            homeTeamAsVisitorDf['HOME_FTA'],
            homeTeamAsVisitorDf['HOME_FT%'],
            homeTeamAsVisitorDf['HOME_ORB'],
            homeTeamAsVisitorDf['HOME_DRB'],
            homeTeamAsVisitorDf['HOME_TRB'],
            homeTeamAsVisitorDf['HOME_AST'],
            homeTeamAsVisitorDf['HOME_STL'],
            homeTeamAsVisitorDf['HOME_BLK'],
            homeTeamAsVisitorDf['HOME_TOV'],
            homeTeamAsVisitorDf['HOME_PF'],
            homeTeamAsVisitorDf['HOME_PTS'],
            homeTeamAsVisitorDf['HOME_TS%'],
            homeTeamAsVisitorDf['HOME_eFG%'],
            homeTeamAsVisitorDf['HOME_3PAr'],
            homeTeamAsVisitorDf['HOME_FTr'],
            homeTeamAsVisitorDf['HOME_ORB%'],
            homeTeamAsVisitorDf['HOME_DRB%'],
            homeTeamAsVisitorDf['HOME_TRB%'],
            homeTeamAsVisitorDf['HOME_AST%'], 
            homeTeamAsVisitorDf['HOME_STL%'], 
            homeTeamAsVisitorDf['HOME_BLK%'],
            homeTeamAsVisitorDf['HOME_TOV%'],
            homeTeamAsVisitorDf['HOME_USG%'],
            homeTeamAsVisitorDf['HOME_ORtg'],
            homeTeamAsVisitorDf['HOME_DRtg'],


            #LAST 6 GAMES

            #Totals visitor as visitor

            last6visitorTeamAsVisitorDf['VISITOR_FG'], 
            last6visitorTeamAsVisitorDf['VISITOR_FGA'],
            last6visitorTeamAsVisitorDf['VISITOR_FG%'],
            last6visitorTeamAsVisitorDf['VISITOR_3P'],
            last6visitorTeamAsVisitorDf['VISITOR_3PA'],
            last6visitorTeamAsVisitorDf['VISITOR_3P%'],
            last6visitorTeamAsVisitorDf['VISITOR_FT'],
            last6visitorTeamAsVisitorDf['VISITOR_FTA'],
            last6visitorTeamAsVisitorDf['VISITOR_FT%'],
            last6visitorTeamAsVisitorDf['VISITOR_ORB'],
            last6visitorTeamAsVisitorDf['VISITOR_DRB'],
            last6visitorTeamAsVisitorDf['VISITOR_TRB'],
            last6visitorTeamAsVisitorDf['VISITOR_AST'],
            last6visitorTeamAsVisitorDf['VISITOR_STL'],
            last6visitorTeamAsVisitorDf['VISITOR_BLK'],
            last6visitorTeamAsVisitorDf['VISITOR_TOV'],
            last6visitorTeamAsVisitorDf['VISITOR_PF'],
            last6visitorTeamAsVisitorDf['VISITOR_PTS'],
            last6visitorTeamAsVisitorDf['VISITOR_TS%'],
            last6visitorTeamAsVisitorDf['VISITOR_eFG%'],
            last6visitorTeamAsVisitorDf['VISITOR_3PAr'],
            last6visitorTeamAsVisitorDf['VISITOR_FTr'],
            last6visitorTeamAsVisitorDf['VISITOR_ORB%'],
            last6visitorTeamAsVisitorDf['VISITOR_DRB%'],
            last6visitorTeamAsVisitorDf['VISITOR_TRB%'],
            last6visitorTeamAsVisitorDf['VISITOR_AST%'], 
            last6visitorTeamAsVisitorDf['VISITOR_STL%'], 
            last6visitorTeamAsVisitorDf['VISITOR_BLK%'],
            last6visitorTeamAsVisitorDf['VISITOR_TOV%'],
            last6visitorTeamAsVisitorDf['VISITOR_USG%'],
            last6visitorTeamAsVisitorDf['VISITOR_ORtg'],
            last6visitorTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against visitor as visitor

            last6visitorTeamAsVisitorDf['HOME_FG'], 
            last6visitorTeamAsVisitorDf['HOME_FGA'],
            last6visitorTeamAsVisitorDf['HOME_FG%'],
            last6visitorTeamAsVisitorDf['HOME_3P'],
            last6visitorTeamAsVisitorDf['HOME_3PA'],
            last6visitorTeamAsVisitorDf['HOME_3P%'],
            last6visitorTeamAsVisitorDf['HOME_FT'],
            last6visitorTeamAsVisitorDf['HOME_FTA'],
            last6visitorTeamAsVisitorDf['HOME_FT%'],
            last6visitorTeamAsVisitorDf['HOME_ORB'],
            last6visitorTeamAsVisitorDf['HOME_DRB'],
            last6visitorTeamAsVisitorDf['HOME_TRB'],
            last6visitorTeamAsVisitorDf['HOME_AST'],
            last6visitorTeamAsVisitorDf['HOME_STL'],
            last6visitorTeamAsVisitorDf['HOME_BLK'],
            last6visitorTeamAsVisitorDf['HOME_TOV'],
            last6visitorTeamAsVisitorDf['HOME_PF'],
            last6visitorTeamAsVisitorDf['HOME_PTS'],
            last6visitorTeamAsVisitorDf['HOME_TS%'],
            last6visitorTeamAsVisitorDf['HOME_eFG%'],
            last6visitorTeamAsVisitorDf['HOME_3PAr'],
            last6visitorTeamAsVisitorDf['HOME_FTr'],
            last6visitorTeamAsVisitorDf['HOME_ORB%'],
            last6visitorTeamAsVisitorDf['HOME_DRB%'],
            last6visitorTeamAsVisitorDf['HOME_TRB%'],
            last6visitorTeamAsVisitorDf['HOME_AST%'], 
            last6visitorTeamAsVisitorDf['HOME_STL%'], 
            last6visitorTeamAsVisitorDf['HOME_BLK%'],
            last6visitorTeamAsVisitorDf['HOME_TOV%'],
            last6visitorTeamAsVisitorDf['HOME_USG%'],
            last6visitorTeamAsVisitorDf['HOME_ORtg'],
            last6visitorTeamAsVisitorDf['HOME_DRtg'],


            #Totals visitor as home

            last6visitorTeamAsHomeDf['HOME_FG'], 
            last6visitorTeamAsHomeDf['HOME_FGA'],
            last6visitorTeamAsHomeDf['HOME_FG%'],
            last6visitorTeamAsHomeDf['HOME_3P'],
            last6visitorTeamAsHomeDf['HOME_3PA'],
            last6visitorTeamAsHomeDf['HOME_3P%'],
            last6visitorTeamAsHomeDf['HOME_FT'],
            last6visitorTeamAsHomeDf['HOME_FTA'],
            last6visitorTeamAsHomeDf['HOME_FT%'],
            last6visitorTeamAsHomeDf['HOME_ORB'],
            last6visitorTeamAsHomeDf['HOME_DRB'],
            last6visitorTeamAsHomeDf['HOME_TRB'],
            last6visitorTeamAsHomeDf['HOME_AST'],
            last6visitorTeamAsHomeDf['HOME_STL'],
            last6visitorTeamAsHomeDf['HOME_BLK'],
            last6visitorTeamAsHomeDf['HOME_TOV'],
            last6visitorTeamAsHomeDf['HOME_PF'],
            last6visitorTeamAsHomeDf['HOME_PTS'],
            last6visitorTeamAsHomeDf['HOME_TS%'],
            last6visitorTeamAsHomeDf['HOME_eFG%'],
            last6visitorTeamAsHomeDf['HOME_3PAr'],
            last6visitorTeamAsHomeDf['HOME_FTr'],
            last6visitorTeamAsHomeDf['HOME_ORB%'],
            last6visitorTeamAsHomeDf['HOME_DRB%'],
            last6visitorTeamAsHomeDf['HOME_TRB%'],
            last6visitorTeamAsHomeDf['HOME_AST%'], 
            last6visitorTeamAsHomeDf['HOME_STL%'], 
            last6visitorTeamAsHomeDf['HOME_BLK%'],
            last6visitorTeamAsHomeDf['HOME_TOV%'],
            last6visitorTeamAsHomeDf['HOME_USG%'],
            last6visitorTeamAsHomeDf['HOME_ORtg'],
            last6visitorTeamAsHomeDf['HOME_DRtg'],

            #Totals against visitor as home

            last6visitorTeamAsHomeDf['VISITOR_FG'], 
            last6visitorTeamAsHomeDf['VISITOR_FGA'],
            last6visitorTeamAsHomeDf['VISITOR_FG%'],
            last6visitorTeamAsHomeDf['VISITOR_3P'],
            last6visitorTeamAsHomeDf['VISITOR_3PA'],
            last6visitorTeamAsHomeDf['VISITOR_3P%'],
            last6visitorTeamAsHomeDf['VISITOR_FT'],
            last6visitorTeamAsHomeDf['VISITOR_FTA'],
            last6visitorTeamAsHomeDf['VISITOR_FT%'],
            last6visitorTeamAsHomeDf['VISITOR_ORB'],
            last6visitorTeamAsHomeDf['VISITOR_DRB'],
            last6visitorTeamAsHomeDf['VISITOR_TRB'],
            last6visitorTeamAsHomeDf['VISITOR_AST'],
            last6visitorTeamAsHomeDf['VISITOR_STL'],
            last6visitorTeamAsHomeDf['VISITOR_BLK'],
            last6visitorTeamAsHomeDf['VISITOR_TOV'],
            last6visitorTeamAsHomeDf['VISITOR_PF'],
            last6visitorTeamAsHomeDf['VISITOR_PTS'],
            last6visitorTeamAsHomeDf['VISITOR_TS%'],
            last6visitorTeamAsHomeDf['VISITOR_eFG%'],
            last6visitorTeamAsHomeDf['VISITOR_3PAr'],
            last6visitorTeamAsHomeDf['VISITOR_FTr'],
            last6visitorTeamAsHomeDf['VISITOR_ORB%'],
            last6visitorTeamAsHomeDf['VISITOR_DRB%'],
            last6visitorTeamAsHomeDf['VISITOR_TRB%'],
            last6visitorTeamAsHomeDf['VISITOR_AST%'], 
            last6visitorTeamAsHomeDf['VISITOR_STL%'], 
            last6visitorTeamAsHomeDf['VISITOR_BLK%'],
            last6visitorTeamAsHomeDf['VISITOR_TOV%'],
            last6visitorTeamAsHomeDf['VISITOR_USG%'],
            last6visitorTeamAsHomeDf['VISITOR_ORtg'],
            last6visitorTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as home

            last6homeTeamAsHomeDf['HOME_FG'], 
            last6homeTeamAsHomeDf['HOME_FGA'],
            last6homeTeamAsHomeDf['HOME_FG%'],
            last6homeTeamAsHomeDf['HOME_3P'],
            last6homeTeamAsHomeDf['HOME_3PA'],
            last6homeTeamAsHomeDf['HOME_3P%'],
            last6homeTeamAsHomeDf['HOME_FT'],
            last6homeTeamAsHomeDf['HOME_FTA'],
            last6homeTeamAsHomeDf['HOME_FT%'],
            last6homeTeamAsHomeDf['HOME_ORB'],
            last6homeTeamAsHomeDf['HOME_DRB'],
            last6homeTeamAsHomeDf['HOME_TRB'],
            last6homeTeamAsHomeDf['HOME_AST'],
            last6homeTeamAsHomeDf['HOME_STL'],
            last6homeTeamAsHomeDf['HOME_BLK'],
            last6homeTeamAsHomeDf['HOME_TOV'],
            last6homeTeamAsHomeDf['HOME_PF'],
            last6homeTeamAsHomeDf['HOME_PTS'],
            last6homeTeamAsHomeDf['HOME_TS%'],
            last6homeTeamAsHomeDf['HOME_eFG%'],
            last6homeTeamAsHomeDf['HOME_3PAr'],
            last6homeTeamAsHomeDf['HOME_FTr'],
            last6homeTeamAsHomeDf['HOME_ORB%'],
            last6homeTeamAsHomeDf['HOME_DRB%'],
            last6homeTeamAsHomeDf['HOME_TRB%'],
            last6homeTeamAsHomeDf['HOME_AST%'], 
            last6homeTeamAsHomeDf['HOME_STL%'], 
            last6homeTeamAsHomeDf['HOME_BLK%'],
            last6homeTeamAsHomeDf['HOME_TOV%'],
            last6homeTeamAsHomeDf['HOME_USG%'],
            last6homeTeamAsHomeDf['HOME_ORtg'],
            last6homeTeamAsHomeDf['HOME_DRtg'],


            #Totals against home as home

            last6homeTeamAsHomeDf['VISITOR_FG'], 
            last6homeTeamAsHomeDf['VISITOR_FGA'],
            last6homeTeamAsHomeDf['VISITOR_FG%'],
            last6homeTeamAsHomeDf['VISITOR_3P'],
            last6homeTeamAsHomeDf['VISITOR_3PA'],
            last6homeTeamAsHomeDf['VISITOR_3P%'],
            last6homeTeamAsHomeDf['VISITOR_FT'],
            last6homeTeamAsHomeDf['VISITOR_FTA'],
            last6homeTeamAsHomeDf['VISITOR_FT%'],
            last6homeTeamAsHomeDf['VISITOR_ORB'],
            last6homeTeamAsHomeDf['VISITOR_DRB'],
            last6homeTeamAsHomeDf['VISITOR_TRB'],
            last6homeTeamAsHomeDf['VISITOR_AST'],
            last6homeTeamAsHomeDf['VISITOR_STL'],
            last6homeTeamAsHomeDf['VISITOR_BLK'],
            last6homeTeamAsHomeDf['VISITOR_TOV'],
            last6homeTeamAsHomeDf['VISITOR_PF'],
            last6homeTeamAsHomeDf['VISITOR_PTS'],
            last6homeTeamAsHomeDf['VISITOR_TS%'],
            last6homeTeamAsHomeDf['VISITOR_eFG%'],
            last6homeTeamAsHomeDf['VISITOR_3PAr'],
            last6homeTeamAsHomeDf['VISITOR_FTr'],
            last6homeTeamAsHomeDf['VISITOR_ORB%'],
            last6homeTeamAsHomeDf['VISITOR_DRB%'],
            last6homeTeamAsHomeDf['VISITOR_TRB%'],
            last6homeTeamAsHomeDf['VISITOR_AST%'], 
            last6homeTeamAsHomeDf['VISITOR_STL%'], 
            last6homeTeamAsHomeDf['VISITOR_BLK%'],
            last6homeTeamAsHomeDf['VISITOR_TOV%'],
            last6homeTeamAsHomeDf['VISITOR_USG%'],
            last6homeTeamAsHomeDf['VISITOR_ORtg'],
            last6homeTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as visitor

            last6homeTeamAsVisitorDf['VISITOR_FG'], 
            last6homeTeamAsVisitorDf['VISITOR_FGA'],
            last6homeTeamAsVisitorDf['VISITOR_FG%'],
            last6homeTeamAsVisitorDf['VISITOR_3P'],
            last6homeTeamAsVisitorDf['VISITOR_3PA'],
            last6homeTeamAsVisitorDf['VISITOR_3P%'],
            last6homeTeamAsVisitorDf['VISITOR_FT'],
            last6homeTeamAsVisitorDf['VISITOR_FTA'],
            last6homeTeamAsVisitorDf['VISITOR_FT%'],
            last6homeTeamAsVisitorDf['VISITOR_ORB'],
            last6homeTeamAsVisitorDf['VISITOR_DRB'],
            last6homeTeamAsVisitorDf['VISITOR_TRB'],
            last6homeTeamAsVisitorDf['VISITOR_AST'],
            last6homeTeamAsVisitorDf['VISITOR_STL'],
            last6homeTeamAsVisitorDf['VISITOR_BLK'],
            last6homeTeamAsVisitorDf['VISITOR_TOV'],
            last6homeTeamAsVisitorDf['VISITOR_PF'],
            last6homeTeamAsVisitorDf['VISITOR_PTS'],
            last6homeTeamAsVisitorDf['VISITOR_TS%'],
            last6homeTeamAsVisitorDf['VISITOR_eFG%'],
            last6homeTeamAsVisitorDf['VISITOR_3PAr'],
            last6homeTeamAsVisitorDf['VISITOR_FTr'],
            last6homeTeamAsVisitorDf['VISITOR_ORB%'],
            last6homeTeamAsVisitorDf['VISITOR_DRB%'],
            last6homeTeamAsVisitorDf['VISITOR_TRB%'],
            last6homeTeamAsVisitorDf['VISITOR_AST%'], 
            last6homeTeamAsVisitorDf['VISITOR_STL%'], 
            last6homeTeamAsVisitorDf['VISITOR_BLK%'],
            last6homeTeamAsVisitorDf['VISITOR_TOV%'],
            last6homeTeamAsVisitorDf['VISITOR_USG%'],
            last6homeTeamAsVisitorDf['VISITOR_ORtg'],
            last6homeTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against home as visitor

            last6homeTeamAsVisitorDf['HOME_FG'], 
            last6homeTeamAsVisitorDf['HOME_FGA'],
            last6homeTeamAsVisitorDf['HOME_FG%'],
            last6homeTeamAsVisitorDf['HOME_3P'],
            last6homeTeamAsVisitorDf['HOME_3PA'],
            last6homeTeamAsVisitorDf['HOME_3P%'],
            last6homeTeamAsVisitorDf['HOME_FT'],
            last6homeTeamAsVisitorDf['HOME_FTA'],
            last6homeTeamAsVisitorDf['HOME_FT%'],
            last6homeTeamAsVisitorDf['HOME_ORB'],
            last6homeTeamAsVisitorDf['HOME_DRB'],
            last6homeTeamAsVisitorDf['HOME_TRB'],
            last6homeTeamAsVisitorDf['HOME_AST'],
            last6homeTeamAsVisitorDf['HOME_STL'],
            last6homeTeamAsVisitorDf['HOME_BLK'],
            last6homeTeamAsVisitorDf['HOME_TOV'],
            last6homeTeamAsVisitorDf['HOME_PF'],
            last6homeTeamAsVisitorDf['HOME_PTS'],
            last6homeTeamAsVisitorDf['HOME_TS%'],
            last6homeTeamAsVisitorDf['HOME_eFG%'],
            last6homeTeamAsVisitorDf['HOME_3PAr'],
            last6homeTeamAsVisitorDf['HOME_FTr'],
            last6homeTeamAsVisitorDf['HOME_ORB%'],
            last6homeTeamAsVisitorDf['HOME_DRB%'],
            last6homeTeamAsVisitorDf['HOME_TRB%'],
            last6homeTeamAsVisitorDf['HOME_AST%'], 
            last6homeTeamAsVisitorDf['HOME_STL%'], 
            last6homeTeamAsVisitorDf['HOME_BLK%'],
            last6homeTeamAsVisitorDf['HOME_TOV%'],
            last6homeTeamAsVisitorDf['HOME_USG%'],
            last6homeTeamAsVisitorDf['HOME_ORtg'],
            last6homeTeamAsVisitorDf['HOME_DRtg'],



            #LAST 3 GAMES

            #Totals visitor as visitor

            last3visitorTeamAsVisitorDf['VISITOR_FG'], 
            last3visitorTeamAsVisitorDf['VISITOR_FGA'],
            last3visitorTeamAsVisitorDf['VISITOR_FG%'],
            last3visitorTeamAsVisitorDf['VISITOR_3P'],
            last3visitorTeamAsVisitorDf['VISITOR_3PA'],
            last3visitorTeamAsVisitorDf['VISITOR_3P%'],
            last3visitorTeamAsVisitorDf['VISITOR_FT'],
            last3visitorTeamAsVisitorDf['VISITOR_FTA'],
            last3visitorTeamAsVisitorDf['VISITOR_FT%'],
            last3visitorTeamAsVisitorDf['VISITOR_ORB'],
            last3visitorTeamAsVisitorDf['VISITOR_DRB'],
            last3visitorTeamAsVisitorDf['VISITOR_TRB'],
            last3visitorTeamAsVisitorDf['VISITOR_AST'],
            last3visitorTeamAsVisitorDf['VISITOR_STL'],
            last3visitorTeamAsVisitorDf['VISITOR_BLK'],
            last3visitorTeamAsVisitorDf['VISITOR_TOV'],
            last3visitorTeamAsVisitorDf['VISITOR_PF'],
            last3visitorTeamAsVisitorDf['VISITOR_PTS'],
            last3visitorTeamAsVisitorDf['VISITOR_TS%'],
            last3visitorTeamAsVisitorDf['VISITOR_eFG%'],
            last3visitorTeamAsVisitorDf['VISITOR_3PAr'],
            last3visitorTeamAsVisitorDf['VISITOR_FTr'],
            last3visitorTeamAsVisitorDf['VISITOR_ORB%'],
            last3visitorTeamAsVisitorDf['VISITOR_DRB%'],
            last3visitorTeamAsVisitorDf['VISITOR_TRB%'],
            last3visitorTeamAsVisitorDf['VISITOR_AST%'], 
            last3visitorTeamAsVisitorDf['VISITOR_STL%'], 
            last3visitorTeamAsVisitorDf['VISITOR_BLK%'],
            last3visitorTeamAsVisitorDf['VISITOR_TOV%'],
            last3visitorTeamAsVisitorDf['VISITOR_USG%'],
            last3visitorTeamAsVisitorDf['VISITOR_ORtg'],
            last3visitorTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against visitor as visitor

            last3visitorTeamAsVisitorDf['HOME_FG'], 
            last3visitorTeamAsVisitorDf['HOME_FGA'],
            last3visitorTeamAsVisitorDf['HOME_FG%'],
            last3visitorTeamAsVisitorDf['HOME_3P'],
            last3visitorTeamAsVisitorDf['HOME_3PA'],
            last3visitorTeamAsVisitorDf['HOME_3P%'],
            last3visitorTeamAsVisitorDf['HOME_FT'],
            last3visitorTeamAsVisitorDf['HOME_FTA'],
            last3visitorTeamAsVisitorDf['HOME_FT%'],
            last3visitorTeamAsVisitorDf['HOME_ORB'],
            last3visitorTeamAsVisitorDf['HOME_DRB'],
            last3visitorTeamAsVisitorDf['HOME_TRB'],
            last3visitorTeamAsVisitorDf['HOME_AST'],
            last3visitorTeamAsVisitorDf['HOME_STL'],
            last3visitorTeamAsVisitorDf['HOME_BLK'],
            last3visitorTeamAsVisitorDf['HOME_TOV'],
            last3visitorTeamAsVisitorDf['HOME_PF'],
            last3visitorTeamAsVisitorDf['HOME_PTS'],
            last3visitorTeamAsVisitorDf['HOME_TS%'],
            last3visitorTeamAsVisitorDf['HOME_eFG%'],
            last3visitorTeamAsVisitorDf['HOME_3PAr'],
            last3visitorTeamAsVisitorDf['HOME_FTr'],
            last3visitorTeamAsVisitorDf['HOME_ORB%'],
            last3visitorTeamAsVisitorDf['HOME_DRB%'],
            last3visitorTeamAsVisitorDf['HOME_TRB%'],
            last3visitorTeamAsVisitorDf['HOME_AST%'], 
            last3visitorTeamAsVisitorDf['HOME_STL%'], 
            last3visitorTeamAsVisitorDf['HOME_BLK%'],
            last3visitorTeamAsVisitorDf['HOME_TOV%'],
            last3visitorTeamAsVisitorDf['HOME_USG%'],
            last3visitorTeamAsVisitorDf['HOME_ORtg'],
            last3visitorTeamAsVisitorDf['HOME_DRtg'],


            #Totals visitor as home

            last3visitorTeamAsHomeDf['HOME_FG'], 
            last3visitorTeamAsHomeDf['HOME_FGA'],
            last3visitorTeamAsHomeDf['HOME_FG%'],
            last3visitorTeamAsHomeDf['HOME_3P'],
            last3visitorTeamAsHomeDf['HOME_3PA'],
            last3visitorTeamAsHomeDf['HOME_3P%'],
            last3visitorTeamAsHomeDf['HOME_FT'],
            last3visitorTeamAsHomeDf['HOME_FTA'],
            last3visitorTeamAsHomeDf['HOME_FT%'],
            last3visitorTeamAsHomeDf['HOME_ORB'],
            last3visitorTeamAsHomeDf['HOME_DRB'],
            last3visitorTeamAsHomeDf['HOME_TRB'],
            last3visitorTeamAsHomeDf['HOME_AST'],
            last3visitorTeamAsHomeDf['HOME_STL'],
            last3visitorTeamAsHomeDf['HOME_BLK'],
            last3visitorTeamAsHomeDf['HOME_TOV'],
            last3visitorTeamAsHomeDf['HOME_PF'],
            last3visitorTeamAsHomeDf['HOME_PTS'],
            last3visitorTeamAsHomeDf['HOME_TS%'],
            last3visitorTeamAsHomeDf['HOME_eFG%'],
            last3visitorTeamAsHomeDf['HOME_3PAr'],
            last3visitorTeamAsHomeDf['HOME_FTr'],
            last3visitorTeamAsHomeDf['HOME_ORB%'],
            last3visitorTeamAsHomeDf['HOME_DRB%'],
            last3visitorTeamAsHomeDf['HOME_TRB%'],
            last3visitorTeamAsHomeDf['HOME_AST%'], 
            last3visitorTeamAsHomeDf['HOME_STL%'], 
            last3visitorTeamAsHomeDf['HOME_BLK%'],
            last3visitorTeamAsHomeDf['HOME_TOV%'],
            last3visitorTeamAsHomeDf['HOME_USG%'],
            last3visitorTeamAsHomeDf['HOME_ORtg'],
            last3visitorTeamAsHomeDf['HOME_DRtg'],

            #Totals against visitor as home

            last3visitorTeamAsHomeDf['VISITOR_FG'], 
            last3visitorTeamAsHomeDf['VISITOR_FGA'],
            last3visitorTeamAsHomeDf['VISITOR_FG%'],
            last3visitorTeamAsHomeDf['VISITOR_3P'],
            last3visitorTeamAsHomeDf['VISITOR_3PA'],
            last3visitorTeamAsHomeDf['VISITOR_3P%'],
            last3visitorTeamAsHomeDf['VISITOR_FT'],
            last3visitorTeamAsHomeDf['VISITOR_FTA'],
            last3visitorTeamAsHomeDf['VISITOR_FT%'],
            last3visitorTeamAsHomeDf['VISITOR_ORB'],
            last3visitorTeamAsHomeDf['VISITOR_DRB'],
            last3visitorTeamAsHomeDf['VISITOR_TRB'],
            last3visitorTeamAsHomeDf['VISITOR_AST'],
            last3visitorTeamAsHomeDf['VISITOR_STL'],
            last3visitorTeamAsHomeDf['VISITOR_BLK'],
            last3visitorTeamAsHomeDf['VISITOR_TOV'],
            last3visitorTeamAsHomeDf['VISITOR_PF'],
            last3visitorTeamAsHomeDf['VISITOR_PTS'],
            last3visitorTeamAsHomeDf['VISITOR_TS%'],
            last3visitorTeamAsHomeDf['VISITOR_eFG%'],
            last3visitorTeamAsHomeDf['VISITOR_3PAr'],
            last3visitorTeamAsHomeDf['VISITOR_FTr'],
            last3visitorTeamAsHomeDf['VISITOR_ORB%'],
            last3visitorTeamAsHomeDf['VISITOR_DRB%'],
            last3visitorTeamAsHomeDf['VISITOR_TRB%'],
            last3visitorTeamAsHomeDf['VISITOR_AST%'], 
            last3visitorTeamAsHomeDf['VISITOR_STL%'], 
            last3visitorTeamAsHomeDf['VISITOR_BLK%'],
            last3visitorTeamAsHomeDf['VISITOR_TOV%'],
            last3visitorTeamAsHomeDf['VISITOR_USG%'],
            last3visitorTeamAsHomeDf['VISITOR_ORtg'],
            last3visitorTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as home

            last3homeTeamAsHomeDf['HOME_FG'], 
            last3homeTeamAsHomeDf['HOME_FGA'],
            last3homeTeamAsHomeDf['HOME_FG%'],
            last3homeTeamAsHomeDf['HOME_3P'],
            last3homeTeamAsHomeDf['HOME_3PA'],
            last3homeTeamAsHomeDf['HOME_3P%'],
            last3homeTeamAsHomeDf['HOME_FT'],
            last3homeTeamAsHomeDf['HOME_FTA'],
            last3homeTeamAsHomeDf['HOME_FT%'],
            last3homeTeamAsHomeDf['HOME_ORB'],
            last3homeTeamAsHomeDf['HOME_DRB'],
            last3homeTeamAsHomeDf['HOME_TRB'],
            last3homeTeamAsHomeDf['HOME_AST'],
            last3homeTeamAsHomeDf['HOME_STL'],
            last3homeTeamAsHomeDf['HOME_BLK'],
            last3homeTeamAsHomeDf['HOME_TOV'],
            last3homeTeamAsHomeDf['HOME_PF'],
            last3homeTeamAsHomeDf['HOME_PTS'],
            last3homeTeamAsHomeDf['HOME_TS%'],
            last3homeTeamAsHomeDf['HOME_eFG%'],
            last3homeTeamAsHomeDf['HOME_3PAr'],
            last3homeTeamAsHomeDf['HOME_FTr'],
            last3homeTeamAsHomeDf['HOME_ORB%'],
            last3homeTeamAsHomeDf['HOME_DRB%'],
            last3homeTeamAsHomeDf['HOME_TRB%'],
            last3homeTeamAsHomeDf['HOME_AST%'], 
            last3homeTeamAsHomeDf['HOME_STL%'], 
            last3homeTeamAsHomeDf['HOME_BLK%'],
            last3homeTeamAsHomeDf['HOME_TOV%'],
            last3homeTeamAsHomeDf['HOME_USG%'],
            last3homeTeamAsHomeDf['HOME_ORtg'],
            last3homeTeamAsHomeDf['HOME_DRtg'],


            #Totals against home as home

            last3homeTeamAsHomeDf['VISITOR_FG'], 
            last3homeTeamAsHomeDf['VISITOR_FGA'],
            last3homeTeamAsHomeDf['VISITOR_FG%'],
            last3homeTeamAsHomeDf['VISITOR_3P'],
            last3homeTeamAsHomeDf['VISITOR_3PA'],
            last3homeTeamAsHomeDf['VISITOR_3P%'],
            last3homeTeamAsHomeDf['VISITOR_FT'],
            last3homeTeamAsHomeDf['VISITOR_FTA'],
            last3homeTeamAsHomeDf['VISITOR_FT%'],
            last3homeTeamAsHomeDf['VISITOR_ORB'],
            last3homeTeamAsHomeDf['VISITOR_DRB'],
            last3homeTeamAsHomeDf['VISITOR_TRB'],
            last3homeTeamAsHomeDf['VISITOR_AST'],
            last3homeTeamAsHomeDf['VISITOR_STL'],
            last3homeTeamAsHomeDf['VISITOR_BLK'],
            last3homeTeamAsHomeDf['VISITOR_TOV'],
            last3homeTeamAsHomeDf['VISITOR_PF'],
            last3homeTeamAsHomeDf['VISITOR_PTS'],
            last3homeTeamAsHomeDf['VISITOR_TS%'],
            last3homeTeamAsHomeDf['VISITOR_eFG%'],
            last3homeTeamAsHomeDf['VISITOR_3PAr'],
            last3homeTeamAsHomeDf['VISITOR_FTr'],
            last3homeTeamAsHomeDf['VISITOR_ORB%'],
            last3homeTeamAsHomeDf['VISITOR_DRB%'],
            last3homeTeamAsHomeDf['VISITOR_TRB%'],
            last3homeTeamAsHomeDf['VISITOR_AST%'], 
            last3homeTeamAsHomeDf['VISITOR_STL%'], 
            last3homeTeamAsHomeDf['VISITOR_BLK%'],
            last3homeTeamAsHomeDf['VISITOR_TOV%'],
            last3homeTeamAsHomeDf['VISITOR_USG%'],
            last3homeTeamAsHomeDf['VISITOR_ORtg'],
            last3homeTeamAsHomeDf['VISITOR_DRtg'],


            #Totals home as visitor

            last3homeTeamAsVisitorDf['VISITOR_FG'], 
            last3homeTeamAsVisitorDf['VISITOR_FGA'],
            last3homeTeamAsVisitorDf['VISITOR_FG%'],
            last3homeTeamAsVisitorDf['VISITOR_3P'],
            last3homeTeamAsVisitorDf['VISITOR_3PA'],
            last3homeTeamAsVisitorDf['VISITOR_3P%'],
            last3homeTeamAsVisitorDf['VISITOR_FT'],
            last3homeTeamAsVisitorDf['VISITOR_FTA'],
            last3homeTeamAsVisitorDf['VISITOR_FT%'],
            last3homeTeamAsVisitorDf['VISITOR_ORB'],
            last3homeTeamAsVisitorDf['VISITOR_DRB'],
            last3homeTeamAsVisitorDf['VISITOR_TRB'],
            last3homeTeamAsVisitorDf['VISITOR_AST'],
            last3homeTeamAsVisitorDf['VISITOR_STL'],
            last3homeTeamAsVisitorDf['VISITOR_BLK'],
            last3homeTeamAsVisitorDf['VISITOR_TOV'],
            last3homeTeamAsVisitorDf['VISITOR_PF'],
            last3homeTeamAsVisitorDf['VISITOR_PTS'],
            last3homeTeamAsVisitorDf['VISITOR_TS%'],
            last3homeTeamAsVisitorDf['VISITOR_eFG%'],
            last3homeTeamAsVisitorDf['VISITOR_3PAr'],
            last3homeTeamAsVisitorDf['VISITOR_FTr'],
            last3homeTeamAsVisitorDf['VISITOR_ORB%'],
            last3homeTeamAsVisitorDf['VISITOR_DRB%'],
            last3homeTeamAsVisitorDf['VISITOR_TRB%'],
            last3homeTeamAsVisitorDf['VISITOR_AST%'], 
            last3homeTeamAsVisitorDf['VISITOR_STL%'], 
            last3homeTeamAsVisitorDf['VISITOR_BLK%'],
            last3homeTeamAsVisitorDf['VISITOR_TOV%'],
            last3homeTeamAsVisitorDf['VISITOR_USG%'],
            last3homeTeamAsVisitorDf['VISITOR_ORtg'],
            last3homeTeamAsVisitorDf['VISITOR_DRtg'],


            #Totals against home as visitor

            last3homeTeamAsVisitorDf['HOME_FG'], 
            last3homeTeamAsVisitorDf['HOME_FGA'],
            last3homeTeamAsVisitorDf['HOME_FG%'],
            last3homeTeamAsVisitorDf['HOME_3P'],
            last3homeTeamAsVisitorDf['HOME_3PA'],
            last3homeTeamAsVisitorDf['HOME_3P%'],
            last3homeTeamAsVisitorDf['HOME_FT'],
            last3homeTeamAsVisitorDf['HOME_FTA'],
            last3homeTeamAsVisitorDf['HOME_FT%'],
            last3homeTeamAsVisitorDf['HOME_ORB'],
            last3homeTeamAsVisitorDf['HOME_DRB'],
            last3homeTeamAsVisitorDf['HOME_TRB'],
            last3homeTeamAsVisitorDf['HOME_AST'],
            last3homeTeamAsVisitorDf['HOME_STL'],
            last3homeTeamAsVisitorDf['HOME_BLK'],
            last3homeTeamAsVisitorDf['HOME_TOV'],
            last3homeTeamAsVisitorDf['HOME_PF'],
            last3homeTeamAsVisitorDf['HOME_PTS'],
            last3homeTeamAsVisitorDf['HOME_TS%'],
            last3homeTeamAsVisitorDf['HOME_eFG%'],
            last3homeTeamAsVisitorDf['HOME_3PAr'],
            last3homeTeamAsVisitorDf['HOME_FTr'],
            last3homeTeamAsVisitorDf['HOME_ORB%'],
            last3homeTeamAsVisitorDf['HOME_DRB%'],
            last3homeTeamAsVisitorDf['HOME_TRB%'],
            last3homeTeamAsVisitorDf['HOME_AST%'], 
            last3homeTeamAsVisitorDf['HOME_STL%'], 
            last3homeTeamAsVisitorDf['HOME_BLK%'],
            last3homeTeamAsVisitorDf['HOME_TOV%'],
            last3homeTeamAsVisitorDf['HOME_USG%'],
            last3homeTeamAsVisitorDf['HOME_ORtg'],
            last3homeTeamAsVisitorDf['HOME_DRtg'],

            row.VISITOR_POINTS + row.HOME_POINTS
        ])


In [33]:
columns = ["VISITOR_TEAM", "HOME_TEAM", 

"VISITOR_AS_VISITOR_FG", "VISITOR_AS_VISITOR_FGA", "VISITOR_AS_VISITOR_FG%", "VISITOR_AS_VISITOR_3P", "VISITOR_AS_VISITOR_3PA", "VISITOR_AS_VISITOR_3P%", "VISITOR_AS_VISITOR_FT", "VISITOR_AS_VISITOR_FTA", "VISITOR_AS_VISITOR_FT%", "VISITOR_AS_VISITOR_ORB", "VISITOR_AS_VISITOR_DRB", "VISITOR_AS_VISITOR_TRB", "VISITOR_AS_VISITOR_AST", "VISITOR_AS_VISITOR_STL", "VISITOR_AS_VISITOR_BLK", "VISITOR_AS_VISITOR_TOV", "VISITOR_AS_VISITOR_PF", "VISITOR_AS_VISITOR_PTS", "VISITOR_AS_VISITOR_TS%", "VISITOR_AS_VISITOR_eFG%", "VISITOR_AS_VISITOR_3PAr", "VISITOR_AS_VISITOR_FTr", "VISITOR_AS_VISITOR_ORB%", "VISITOR_AS_VISITOR_DRB%", "VISITOR_AS_VISITOR_TRB%", "VISITOR_AS_VISITOR_AST%", "VISITOR_AS_VISITOR_STL%", "VISITOR_AS_VISITOR_BLK%", "VISITOR_AS_VISITOR_TOV%", "VISITOR_AS_VISITOR_USG%", "VISITOR_AS_VISITOR_ORtg", "VISITOR_AS_VISITOR_DRtg",

"VISITOR_AS_VISITOR_AGAINST_FG", "VISITOR_AS_VISITOR_AGAINST_FGA", "VISITOR_AS_VISITOR_AGAINST_FG%", "VISITOR_AS_VISITOR_AGAINST_3P", "VISITOR_AS_VISITOR_AGAINST_3PA", "VISITOR_AS_VISITOR_AGAINST_3P%", "VISITOR_AS_VISITOR_AGAINST_FT", "VISITOR_AS_VISITOR_AGAINST_FTA", "VISITOR_AS_VISITOR_AGAINST_FT%", "VISITOR_AS_VISITOR_AGAINST_ORB", "VISITOR_AS_VISITOR_AGAINST_DRB", "VISITOR_AS_VISITOR_AGAINST_TRB", "VISITOR_AS_VISITOR_AGAINST_AST", "VISITOR_AS_VISITOR_AGAINST_STL", "VISITOR_AS_VISITOR_AGAINST_BLK", "VISITOR_AS_VISITOR_AGAINST_TOV", "VISITOR_AS_VISITOR_AGAINST_PF", "VISITOR_AS_VISITOR_AGAINST_PTS", "VISITOR_AS_VISITOR_AGAINST_TS%", "VISITOR_AS_VISITOR_AGAINST_eFG%", "VISITOR_AS_VISITOR_AGAINST_3PAr", "VISITOR_AS_VISITOR_AGAINST_FTr", "VISITOR_AS_VISITOR_AGAINST_ORB%", "VISITOR_AS_VISITOR_AGAINST_DRB%", "VISITOR_AS_VISITOR_AGAINST_TRB%", "VISITOR_AS_VISITOR_AGAINST_AST%", "VISITOR_AS_VISITOR_AGAINST_STL%", "VISITOR_AS_VISITOR_AGAINST_BLK%", "VISITOR_AS_VISITOR_AGAINST_TOV%", "VISITOR_AS_VISITOR_AGAINST_USG%", "VISITOR_AS_VISITOR_AGAINST_ORtg", "VISITOR_AS_VISITOR_AGAINST_DRtg",

"VISITOR_AS_HOME_FG", "VISITOR_AS_HOME_FGA", "VISITOR_AS_HOME_FG%", "VISITOR_AS_HOME_3P", "VISITOR_AS_HOME_3PA", "VISITOR_AS_HOME_3P%", "VISITOR_AS_HOME_FT", "VISITOR_AS_HOME_FTA", "VISITOR_AS_HOME_FT%", "VISITOR_AS_HOME_ORB", "VISITOR_AS_HOME_DRB", "VISITOR_AS_HOME_TRB", "VISITOR_AS_HOME_AST", "VISITOR_AS_HOME_STL", "VISITOR_AS_HOME_BLK", "VISITOR_AS_HOME_TOV", "VISITOR_AS_HOME_PF", "VISITOR_AS_HOME_PTS", "VISITOR_AS_HOME_TS%", "VISITOR_AS_HOME_eFG%", "VISITOR_AS_HOME_3PAr", "VISITOR_AS_HOME_FTr", "VISITOR_AS_HOME_ORB%", "VISITOR_AS_HOME_DRB%", "VISITOR_AS_HOME_TRB%", "VISITOR_AS_HOME_AST%", "VISITOR_AS_HOME_STL%", "VISITOR_AS_HOME_BLK%", "VISITOR_AS_HOME_TOV%", "VISITOR_AS_HOME_USG%", "VISITOR_AS_HOME_ORtg", "VISITOR_AS_HOME_DRtg",

"VISITOR_AS_HOME_AGAINST_FG", "VISITOR_AS_HOME_AGAINST_FGA", "VISITOR_AS_HOME_AGAINST_FG%", "VISITOR_AS_HOME_AGAINST_3P", "VISITOR_AS_HOME_AGAINST_3PA", "VISITOR_AS_HOME_AGAINST_3P%", "VISITOR_AS_HOME_AGAINST_FT", "VISITOR_AS_HOME_AGAINST_FTA", "VISITOR_AS_HOME_AGAINST_FT%", "VISITOR_AS_HOME_AGAINST_ORB", "VISITOR_AS_HOME_AGAINST_DRB", "VISITOR_AS_HOME_AGAINST_TRB", "VISITOR_AS_HOME_AGAINST_AST", "VISITOR_AS_HOME_AGAINST_STL", "VISITOR_AS_HOME_AGAINST_BLK", "VISITOR_AS_HOME_AGAINST_TOV", "VISITOR_AS_HOME_AGAINST_PF", "VISITOR_AS_HOME_AGAINST_PTS", "VISITOR_AS_HOME_AGAINST_TS%", "VISITOR_AS_HOME_AGAINST_eFG%", "VISITOR_AS_HOME_AGAINST_3PAr", "VISITOR_AS_HOME_AGAINST_FTr", "VISITOR_AS_HOME_AGAINST_ORB%", "VISITOR_AS_HOME_AGAINST_DRB%", "VISITOR_AS_HOME_AGAINST_TRB%", "VISITOR_AS_HOME_AGAINST_AST%", "VISITOR_AS_HOME_AGAINST_STL%", "VISITOR_AS_HOME_AGAINST_BLK%", "VISITOR_AS_HOME_AGAINST_TOV%", "VISITOR_AS_HOME_AGAINST_USG%", "VISITOR_AS_HOME_AGAINST_ORtg", "VISITOR_AS_HOME_AGAINST_DRtg",

"HOME_AS_HOME_FG", "HOME_AS_HOME_FGA", "HOME_AS_HOME_FG%", "HOME_AS_HOME_3P", "HOME_AS_HOME_3PA", "HOME_AS_HOME_3P%", "HOME_AS_HOME_FT", "HOME_AS_HOME_FTA", "HOME_AS_HOME_FT%", "HOME_AS_HOME_ORB", "HOME_AS_HOME_DRB", "HOME_AS_HOME_TRB", "HOME_AS_HOME_AST", "HOME_AS_HOME_STL", "HOME_AS_HOME_BLK", "HOME_AS_HOME_TOV", "HOME_AS_HOME_PF", "HOME_AS_HOME_PTS", "HOME_AS_HOME_TS%", "HOME_AS_HOME_eFG%", "HOME_AS_HOME_3PAr", "HOME_AS_HOME_FTr", "HOME_AS_HOME_ORB%", "HOME_AS_HOME_DRB%", "HOME_AS_HOME_TRB%", "HOME_AS_HOME_AST%", "HOME_AS_HOME_STL%", "HOME_AS_HOME_BLK%", "HOME_AS_HOME_TOV%", "HOME_AS_HOME_USG%", "HOME_AS_HOME_ORtg", "HOME_AS_HOME_DRtg",

"HOME_AS_HOME_AGAINST_FG", "HOME_AS_HOME_AGAINST_FGA", "HOME_AS_HOME_AGAINST_FG%", "HOME_AS_HOME_AGAINST_3P", "HOME_AS_HOME_AGAINST_3PA", "HOME_AS_HOME_AGAINST_3P%", "HOME_AS_HOME_AGAINST_FT", "HOME_AS_HOME_AGAINST_FTA", "HOME_AS_HOME_AGAINST_FT%", "HOME_AS_HOME_AGAINST_ORB", "HOME_AS_HOME_AGAINST_DRB", "HOME_AS_HOME_AGAINST_TRB", "HOME_AS_HOME_AGAINST_AST", "HOME_AS_HOME_AGAINST_STL", "HOME_AS_HOME_AGAINST_BLK", "HOME_AS_HOME_AGAINST_TOV", "HOME_AS_HOME_AGAINST_PF", "HOME_AS_HOME_AGAINST_PTS", "HOME_AS_HOME_AGAINST_TS%", "HOME_AS_HOME_AGAINST_eFG%", "HOME_AS_HOME_AGAINST_3PAr", "HOME_AS_HOME_AGAINST_FTr", "HOME_AS_HOME_AGAINST_ORB%", "HOME_AS_HOME_AGAINST_DRB%", "HOME_AS_HOME_AGAINST_TRB%", "HOME_AS_HOME_AGAINST_AST%", "HOME_AS_HOME_AGAINST_STL%", "HOME_AS_HOME_AGAINST_BLK%", "HOME_AS_HOME_AGAINST_TOV%", "HOME_AS_HOME_AGAINST_USG%", "HOME_AS_HOME_AGAINST_ORtg", "HOME_AS_HOME_AGAINST_DRtg",

"HOME_AS_VISITOR_FG", "HOME_AS_VISITOR_FGA", "HOME_AS_VISITOR_FG%", "HOME_AS_VISITOR_3P", "HOME_AS_VISITOR_3PA", "HOME_AS_VISITOR_3P%", "HOME_AS_VISITOR_FT", "HOME_AS_VISITOR_FTA", "HOME_AS_VISITOR_FT%", "HOME_AS_VISITOR_ORB", "HOME_AS_VISITOR_DRB", "HOME_AS_VISITOR_TRB", "HOME_AS_HOME_AST", "HOME_AS_VISITOR_STL", "HOME_AS_VISITOR_BLK", "HOME_AS_VISITOR_TOV", "HOME_AS_VISITOR_PF", "HOME_AS_VISITOR_PTS", "HOME_AS_VISITOR_TS%", "HOME_AS_VISITOR_eFG%", "HOME_AS_VISITOR_3PAr", "HOME_AS_VISITOR_FTr", "HOME_AS_VISITOR_ORB%", "HOME_AS_VISITOR_DRB%", "HOME_AS_VISITOR_TRB%", "HOME_AS_HOME_AST%", "HOME_AS_VISITOR_STL%", "HOME_AS_VISITOR_BLK%", "HOME_AS_VISITOR_TOV%", "HOME_AS_VISITOR_USG%", "HOME_AS_VISITOR_ORtg", "HOME_AS_VISITOR_DRtg",

"HOME_AS_VISITOR_AGAINST_FG", "HOME_AS_VISITOR_AGAINST_FGA", "HOME_AS_VISITOR_AGAINST_FG%", "HOME_AS_VISITOR_AGAINST_3P", "HOME_AS_VISITOR_AGAINST_3PA", "HOME_AS_VISITOR_AGAINST_3P%", "HOME_AS_VISITOR_AGAINST_FT", "HOME_AS_VISITOR_AGAINST_FTA", "HOME_AS_VISITOR_AGAINST_FT%", "HOME_AS_VISITOR_AGAINST_ORB", "HOME_AS_VISITOR_AGAINST_DRB", "HOME_AS_VISITOR_AGAINST_TRB", "HOME_AS_VISITOR_AGAINST_AST", "HOME_AS_VISITOR_AGAINST_STL", "HOME_AS_VISITOR_AGAINST_BLK", "HOME_AS_VISITOR_AGAINST_TOV", "HOME_AS_VISITOR_AGAINST_PF", "HOME_AS_VISITOR_AGAINST_PTS", "HOME_AS_VISITOR_AGAINST_TS%", "HOME_AS_VISITOR_AGAINST_eFG%", "HOME_AS_VISITOR_AGAINST_3PAr", "HOME_AS_VISITOR_AGAINST_FTr", "HOME_AS_VISITOR_AGAINST_ORB%", "HOME_AS_VISITOR_AGAINST_DRB%", "HOME_AS_VISITOR_AGAINST_TRB%", "HOME_AS_VISITOR_AGAINST_AST%", "HOME_AS_VISITOR_AGAINST_STL%", "HOME_AS_VISITOR_AGAINST_BLK%", "HOME_AS_VISITOR_AGAINST_TOV%", "HOME_AS_VISITOR_AGAINST_USG%", "HOME_AS_VISITOR_AGAINST_ORtg", "HOME_AS_VISITOR_AGAINST_DRtg",


"LAST_6_VISITOR_AS_VISITOR_FG", "LAST_6_VISITOR_AS_VISITOR_FGA", "LAST_6_VISITOR_AS_VISITOR_FG%", "LAST_6_VISITOR_AS_VISITOR_3P", "LAST_6_VISITOR_AS_VISITOR_3PA", "LAST_6_VISITOR_AS_VISITOR_3P%", "LAST_6_VISITOR_AS_VISITOR_FT", "LAST_6_VISITOR_AS_VISITOR_FTA", "LAST_6_VISITOR_AS_VISITOR_FT%", "LAST_6_VISITOR_AS_VISITOR_ORB", "LAST_6_VISITOR_AS_VISITOR_DRB", "LAST_6_VISITOR_AS_VISITOR_TRB", "LAST_6_VISITOR_AS_VISITOR_AST", "LAST_6_VISITOR_AS_VISITOR_STL", "LAST_6_VISITOR_AS_VISITOR_BLK", "LAST_6_VISITOR_AS_VISITOR_TOV", "LAST_6_VISITOR_AS_VISITOR_PF", "LAST_6_VISITOR_AS_VISITOR_PTS", "LAST_6_VISITOR_AS_VISITOR_TS%", "LAST_6_VISITOR_AS_VISITOR_eFG%", "LAST_6_VISITOR_AS_VISITOR_3PAr", "LAST_6_VISITOR_AS_VISITOR_FTr", "LAST_6_VISITOR_AS_VISITOR_ORB%", "LAST_6_VISITOR_AS_VISITOR_DRB%", "LAST_6_VISITOR_AS_VISITOR_TRB%", "LAST_6_VISITOR_AS_VISITOR_AST%", "LAST_6_VISITOR_AS_VISITOR_STL%", "LAST_6_VISITOR_AS_VISITOR_BLK%", "LAST_6_VISITOR_AS_VISITOR_TOV%", "LAST_6_VISITOR_AS_VISITOR_USG%", "LAST_6_VISITOR_AS_VISITOR_ORtg", "LAST_6_VISITOR_AS_VISITOR_DRtg",

"LAST_6_VISITOR_AS_VISITOR_AGAINST_FG", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FGA", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FG%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_3P", "LAST_6_VISITOR_AS_VISITOR_AGAINST_3PA", "LAST_6_VISITOR_AS_VISITOR_AGAINST_3P%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FT", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FTA", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FT%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_ORB", "LAST_6_VISITOR_AS_VISITOR_AGAINST_DRB", "LAST_6_VISITOR_AS_VISITOR_AGAINST_TRB", "LAST_6_VISITOR_AS_VISITOR_AGAINST_AST", "LAST_6_VISITOR_AS_VISITOR_AGAINST_STL", "LAST_6_VISITOR_AS_VISITOR_AGAINST_BLK", "LAST_6_VISITOR_AS_VISITOR_AGAINST_TOV", "LAST_6_VISITOR_AS_VISITOR_AGAINST_PF", "LAST_6_VISITOR_AS_VISITOR_AGAINST_PTS", "LAST_6_VISITOR_AS_VISITOR_AGAINST_TS%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_eFG%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_3PAr", "LAST_6_VISITOR_AS_VISITOR_AGAINST_FTr", "LAST_6_VISITOR_AS_VISITOR_AGAINST_ORB%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_DRB%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_TRB%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_AST%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_STL%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_BLK%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_TOV%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_USG%", "LAST_6_VISITOR_AS_VISITOR_AGAINST_ORtg", "LAST_6_VISITOR_AS_VISITOR_AGAINST_DRtg",

"LAST_6_VISITOR_AS_HOME_FG", "LAST_6_VISITOR_AS_HOME_FGA", "LAST_6_VISITOR_AS_HOME_FG%", "LAST_6_VISITOR_AS_HOME_3P", "LAST_6_VISITOR_AS_HOME_3PA", "LAST_6_VISITOR_AS_HOME_3P%", "LAST_6_VISITOR_AS_HOME_FT", "LAST_6_VISITOR_AS_HOME_FTA", "LAST_6_VISITOR_AS_HOME_FT%", "LAST_6_VISITOR_AS_HOME_ORB", "LAST_6_VISITOR_AS_HOME_DRB", "LAST_6_VISITOR_AS_HOME_TRB", "LAST_6_VISITOR_AS_HOME_AST", "LAST_6_VISITOR_AS_HOME_STL", "LAST_6_VISITOR_AS_HOME_BLK", "LAST_6_VISITOR_AS_HOME_TOV", "LAST_6_VISITOR_AS_HOME_PF", "LAST_6_VISITOR_AS_HOME_PTS", "LAST_6_VISITOR_AS_HOME_TS%", "LAST_6_VISITOR_AS_HOME_eFG%", "LAST_6_VISITOR_AS_HOME_3PAr", "LAST_6_VISITOR_AS_HOME_FTr", "LAST_6_VISITOR_AS_HOME_ORB%", "LAST_6_VISITOR_AS_HOME_DRB%", "LAST_6_VISITOR_AS_HOME_TRB%", "LAST_6_VISITOR_AS_HOME_AST%", "LAST_6_VISITOR_AS_HOME_STL%", "LAST_6_VISITOR_AS_HOME_BLK%", "LAST_6_VISITOR_AS_HOME_TOV%", "LAST_6_VISITOR_AS_HOME_USG%", "LAST_6_VISITOR_AS_HOME_ORtg", "LAST_6_VISITOR_AS_HOME_DRtg",

"LAST_6_VISITOR_AS_HOME_AGAINST_FG", "LAST_6_VISITOR_AS_HOME_AGAINST_FGA", "LAST_6_VISITOR_AS_HOME_AGAINST_FG%", "LAST_6_VISITOR_AS_HOME_AGAINST_3P", "LAST_6_VISITOR_AS_HOME_AGAINST_3PA", "LAST_6_VISITOR_AS_HOME_AGAINST_3P%", "LAST_6_VISITOR_AS_HOME_AGAINST_FT", "LAST_6_VISITOR_AS_HOME_AGAINST_FTA", "LAST_6_VISITOR_AS_HOME_AGAINST_FT%", "LAST_6_VISITOR_AS_HOME_AGAINST_ORB", "LAST_6_VISITOR_AS_HOME_AGAINST_DRB", "LAST_6_VISITOR_AS_HOME_AGAINST_TRB", "LAST_6_VISITOR_AS_HOME_AGAINST_AST", "LAST_6_VISITOR_AS_HOME_AGAINST_STL", "LAST_6_VISITOR_AS_HOME_AGAINST_BLK", "LAST_6_VISITOR_AS_HOME_AGAINST_TOV", "LAST_6_VISITOR_AS_HOME_AGAINST_PF", "LAST_6_VISITOR_AS_HOME_AGAINST_PTS", "LAST_6_VISITOR_AS_HOME_AGAINST_TS%", "LAST_6_VISITOR_AS_HOME_AGAINST_eFG%", "LAST_6_VISITOR_AS_HOME_AGAINST_3PAr", "LAST_6_VISITOR_AS_HOME_AGAINST_FTr", "LAST_6_VISITOR_AS_HOME_AGAINST_ORB%", "LAST_6_VISITOR_AS_HOME_AGAINST_DRB%", "LAST_6_VISITOR_AS_HOME_AGAINST_TRB%", "LAST_6_VISITOR_AS_HOME_AGAINST_AST%", "LAST_6_VISITOR_AS_HOME_AGAINST_STL%", "LAST_6_VISITOR_AS_HOME_AGAINST_BLK%", "LAST_6_VISITOR_AS_HOME_AGAINST_TOV%", "LAST_6_VISITOR_AS_HOME_AGAINST_USG%", "LAST_6_VISITOR_AS_HOME_AGAINST_ORtg", "LAST_6_VISITOR_AS_HOME_AGAINST_DRtg",

"LAST_6_HOME_AS_HOME_FG", "LAST_6_HOME_AS_HOME_FGA", "LAST_6_HOME_AS_HOME_FG%", "LAST_6_HOME_AS_HOME_3P", "LAST_6_HOME_AS_HOME_3PA", "LAST_6_HOME_AS_HOME_3P%", "LAST_6_HOME_AS_HOME_FT", "LAST_6_HOME_AS_HOME_FTA", "LAST_6_HOME_AS_HOME_FT%", "LAST_6_HOME_AS_HOME_ORB", "LAST_6_HOME_AS_HOME_DRB", "LAST_6_HOME_AS_HOME_TRB", "LAST_6_HOME_AS_HOME_AST", "LAST_6_HOME_AS_HOME_STL", "LAST_6_HOME_AS_HOME_BLK", "LAST_6_HOME_AS_HOME_TOV", "LAST_6_HOME_AS_HOME_PF", "LAST_6_HOME_AS_HOME_PTS", "LAST_6_HOME_AS_HOME_TS%", "LAST_6_HOME_AS_HOME_eFG%", "LAST_6_HOME_AS_HOME_3PAr", "LAST_6_HOME_AS_HOME_FTr", "LAST_6_HOME_AS_HOME_ORB%", "LAST_6_HOME_AS_HOME_DRB%", "LAST_6_HOME_AS_HOME_TRB%", "LAST_6_HOME_AS_HOME_AST%", "LAST_6_HOME_AS_HOME_STL%", "LAST_6_HOME_AS_HOME_BLK%", "LAST_6_HOME_AS_HOME_TOV%", "LAST_6_HOME_AS_HOME_USG%", "LAST_6_HOME_AS_HOME_ORtg", "LAST_6_HOME_AS_HOME_DRtg",

"LAST_6_HOME_AS_HOME_AGAINST_FG", "LAST_6_HOME_AS_HOME_AGAINST_FGA", "LAST_6_HOME_AS_HOME_AGAINST_FG%", "LAST_6_HOME_AS_HOME_AGAINST_3P", "LAST_6_HOME_AS_HOME_AGAINST_3PA", "LAST_6_HOME_AS_HOME_AGAINST_3P%", "LAST_6_HOME_AS_HOME_AGAINST_FT", "LAST_6_HOME_AS_HOME_AGAINST_FTA", "LAST_6_HOME_AS_HOME_AGAINST_FT%", "LAST_6_HOME_AS_HOME_AGAINST_ORB", "LAST_6_HOME_AS_HOME_AGAINST_DRB", "LAST_6_HOME_AS_HOME_AGAINST_TRB", "LAST_6_HOME_AS_HOME_AGAINST_AST", "LAST_6_HOME_AS_HOME_AGAINST_STL", "LAST_6_HOME_AS_HOME_AGAINST_BLK", "LAST_6_HOME_AS_HOME_AGAINST_TOV", "LAST_6_HOME_AS_HOME_AGAINST_PF", "LAST_6_HOME_AS_HOME_AGAINST_PTS", "LAST_6_HOME_AS_HOME_AGAINST_TS%", "LAST_6_HOME_AS_HOME_AGAINST_eFG%", "LAST_6_HOME_AS_HOME_AGAINST_3PAr", "LAST_6_HOME_AS_HOME_AGAINST_FTr", "LAST_6_HOME_AS_HOME_AGAINST_ORB%", "LAST_6_HOME_AS_HOME_AGAINST_DRB%", "LAST_6_HOME_AS_HOME_AGAINST_TRB%", "LAST_6_HOME_AS_HOME_AGAINST_AST%", "LAST_6_HOME_AS_HOME_AGAINST_STL%", "LAST_6_HOME_AS_HOME_AGAINST_BLK%", "LAST_6_HOME_AS_HOME_AGAINST_TOV%", "LAST_6_HOME_AS_HOME_AGAINST_USG%", "LAST_6_HOME_AS_HOME_AGAINST_ORtg", "LAST_6_HOME_AS_HOME_AGAINST_DRtg",

"LAST_6_HOME_AS_VISITOR_FG", "LAST_6_HOME_AS_VISITOR_FGA", "LAST_6_HOME_AS_VISITOR_FG%", "LAST_6_HOME_AS_VISITOR_3P", "LAST_6_HOME_AS_VISITOR_3PA", "LAST_6_HOME_AS_VISITOR_3P%", "LAST_6_HOME_AS_VISITOR_FT", "LAST_6_HOME_AS_VISITOR_FTA", "LAST_6_HOME_AS_VISITOR_FT%", "LAST_6_HOME_AS_VISITOR_ORB", "LAST_6_HOME_AS_VISITOR_DRB", "LAST_6_HOME_AS_VISITOR_TRB", "LAST_6_HOME_AS_HOME_AST", "LAST_6_HOME_AS_VISITOR_STL", "LAST_6_HOME_AS_VISITOR_BLK", "LAST_6_HOME_AS_VISITOR_TOV", "LAST_6_HOME_AS_VISITOR_PF", "LAST_6_HOME_AS_VISITOR_PTS", "LAST_6_HOME_AS_VISITOR_TS%", "LAST_6_HOME_AS_VISITOR_eFG%", "LAST_6_HOME_AS_VISITOR_3PAr", "LAST_6_HOME_AS_VISITOR_FTr", "LAST_6_HOME_AS_VISITOR_ORB%", "LAST_6_HOME_AS_VISITOR_DRB%", "LAST_6_HOME_AS_VISITOR_TRB%", "LAST_6_HOME_AS_HOME_AST%", "LAST_6_HOME_AS_VISITOR_STL%", "LAST_6_HOME_AS_VISITOR_BLK%", "LAST_6_HOME_AS_VISITOR_TOV%", "LAST_6_HOME_AS_VISITOR_USG%", "LAST_6_HOME_AS_VISITOR_ORtg", "LAST_6_HOME_AS_VISITOR_DRtg",

"LAST_6_HOME_AS_VISITOR_AGAINST_FG", "LAST_6_HOME_AS_VISITOR_AGAINST_FGA", "LAST_6_HOME_AS_VISITOR_AGAINST_FG%", "LAST_6_HOME_AS_VISITOR_AGAINST_3P", "LAST_6_HOME_AS_VISITOR_AGAINST_3PA", "LAST_6_HOME_AS_VISITOR_AGAINST_3P%", "LAST_6_HOME_AS_VISITOR_AGAINST_FT", "LAST_6_HOME_AS_VISITOR_AGAINST_FTA", "LAST_6_HOME_AS_VISITOR_AGAINST_FT%", "LAST_6_HOME_AS_VISITOR_AGAINST_ORB", "LAST_6_HOME_AS_VISITOR_AGAINST_DRB", "LAST_6_HOME_AS_VISITOR_AGAINST_TRB", "LAST_6_HOME_AS_VISITOR_AGAINST_AST", "LAST_6_HOME_AS_VISITOR_AGAINST_STL", "LAST_6_HOME_AS_VISITOR_AGAINST_BLK", "LAST_6_HOME_AS_VISITOR_AGAINST_TOV", "LAST_6_HOME_AS_VISITOR_AGAINST_PF", "LAST_6_HOME_AS_VISITOR_AGAINST_PTS", "LAST_6_HOME_AS_VISITOR_AGAINST_TS%", "LAST_6_HOME_AS_VISITOR_AGAINST_eFG%", "LAST_6_HOME_AS_VISITOR_AGAINST_3PAr", "LAST_6_HOME_AS_VISITOR_AGAINST_FTr", "LAST_6_HOME_AS_VISITOR_AGAINST_ORB%", "LAST_6_HOME_AS_VISITOR_AGAINST_DRB%", "LAST_6_HOME_AS_VISITOR_AGAINST_TRB%", "LAST_6_HOME_AS_VISITOR_AGAINST_AST%", "LAST_6_HOME_AS_VISITOR_AGAINST_STL%", "LAST_6_HOME_AS_VISITOR_AGAINST_BLK%", "LAST_6_HOME_AS_VISITOR_AGAINST_TOV%", "LAST_6_HOME_AS_VISITOR_AGAINST_USG%", "LAST_6_HOME_AS_VISITOR_AGAINST_ORtg", "LAST_6_HOME_AS_VISITOR_AGAINST_DRtg",


"LAST_3_VISITOR_AS_VISITOR_FG", "LAST_3_VISITOR_AS_VISITOR_FGA", "LAST_3_VISITOR_AS_VISITOR_FG%", "LAST_3_VISITOR_AS_VISITOR_3P", "LAST_3_VISITOR_AS_VISITOR_3PA", "LAST_3_VISITOR_AS_VISITOR_3P%", "LAST_3_VISITOR_AS_VISITOR_FT", "LAST_3_VISITOR_AS_VISITOR_FTA", "LAST_3_VISITOR_AS_VISITOR_FT%", "LAST_3_VISITOR_AS_VISITOR_ORB", "LAST_3_VISITOR_AS_VISITOR_DRB", "LAST_3_VISITOR_AS_VISITOR_TRB", "LAST_3_VISITOR_AS_VISITOR_AST", "LAST_3_VISITOR_AS_VISITOR_STL", "LAST_3_VISITOR_AS_VISITOR_BLK", "LAST_3_VISITOR_AS_VISITOR_TOV", "LAST_3_VISITOR_AS_VISITOR_PF", "LAST_3_VISITOR_AS_VISITOR_PTS", "LAST_3_VISITOR_AS_VISITOR_TS%", "LAST_3_VISITOR_AS_VISITOR_eFG%", "LAST_3_VISITOR_AS_VISITOR_3PAr", "LAST_3_VISITOR_AS_VISITOR_FTr", "LAST_3_VISITOR_AS_VISITOR_ORB%", "LAST_3_VISITOR_AS_VISITOR_DRB%", "LAST_3_VISITOR_AS_VISITOR_TRB%", "LAST_3_VISITOR_AS_VISITOR_AST%", "LAST_3_VISITOR_AS_VISITOR_STL%", "LAST_3_VISITOR_AS_VISITOR_BLK%", "LAST_3_VISITOR_AS_VISITOR_TOV%", "LAST_3_VISITOR_AS_VISITOR_USG%", "LAST_3_VISITOR_AS_VISITOR_ORtg", "LAST_3_VISITOR_AS_VISITOR_DRtg",

"LAST_3_VISITOR_AS_VISITOR_AGAINST_FG", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FGA", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FG%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_3P", "LAST_3_VISITOR_AS_VISITOR_AGAINST_3PA", "LAST_3_VISITOR_AS_VISITOR_AGAINST_3P%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FT", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FTA", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FT%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_ORB", "LAST_3_VISITOR_AS_VISITOR_AGAINST_DRB", "LAST_3_VISITOR_AS_VISITOR_AGAINST_TRB", "LAST_3_VISITOR_AS_VISITOR_AGAINST_AST", "LAST_3_VISITOR_AS_VISITOR_AGAINST_STL", "LAST_3_VISITOR_AS_VISITOR_AGAINST_BLK", "LAST_3_VISITOR_AS_VISITOR_AGAINST_TOV", "LAST_3_VISITOR_AS_VISITOR_AGAINST_PF", "LAST_3_VISITOR_AS_VISITOR_AGAINST_PTS", "LAST_3_VISITOR_AS_VISITOR_AGAINST_TS%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_eFG%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_3PAr", "LAST_3_VISITOR_AS_VISITOR_AGAINST_FTr", "LAST_3_VISITOR_AS_VISITOR_AGAINST_ORB%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_DRB%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_TRB%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_AST%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_STL%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_BLK%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_TOV%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_USG%", "LAST_3_VISITOR_AS_VISITOR_AGAINST_ORtg", "LAST_3_VISITOR_AS_VISITOR_AGAINST_DRtg",

"LAST_3_VISITOR_AS_HOME_FG", "LAST_3_VISITOR_AS_HOME_FGA", "LAST_3_VISITOR_AS_HOME_FG%", "LAST_3_VISITOR_AS_HOME_3P", "LAST_3_VISITOR_AS_HOME_3PA", "LAST_3_VISITOR_AS_HOME_3P%", "LAST_3_VISITOR_AS_HOME_FT", "LAST_3_VISITOR_AS_HOME_FTA", "LAST_3_VISITOR_AS_HOME_FT%", "LAST_3_VISITOR_AS_HOME_ORB", "LAST_3_VISITOR_AS_HOME_DRB", "LAST_3_VISITOR_AS_HOME_TRB", "LAST_3_VISITOR_AS_HOME_AST", "LAST_3_VISITOR_AS_HOME_STL", "LAST_3_VISITOR_AS_HOME_BLK", "LAST_3_VISITOR_AS_HOME_TOV", "LAST_3_VISITOR_AS_HOME_PF", "LAST_3_VISITOR_AS_HOME_PTS", "LAST_3_VISITOR_AS_HOME_TS%", "LAST_3_VISITOR_AS_HOME_eFG%", "LAST_3_VISITOR_AS_HOME_3PAr", "LAST_3_VISITOR_AS_HOME_FTr", "LAST_3_VISITOR_AS_HOME_ORB%", "LAST_3_VISITOR_AS_HOME_DRB%", "LAST_3_VISITOR_AS_HOME_TRB%", "LAST_3_VISITOR_AS_HOME_AST%", "LAST_3_VISITOR_AS_HOME_STL%", "LAST_3_VISITOR_AS_HOME_BLK%", "LAST_3_VISITOR_AS_HOME_TOV%", "LAST_3_VISITOR_AS_HOME_USG%", "LAST_3_VISITOR_AS_HOME_ORtg", "LAST_3_VISITOR_AS_HOME_DRtg",

"LAST_3_VISITOR_AS_HOME_AGAINST_FG", "LAST_3_VISITOR_AS_HOME_AGAINST_FGA", "LAST_3_VISITOR_AS_HOME_AGAINST_FG%", "LAST_3_VISITOR_AS_HOME_AGAINST_3P", "LAST_3_VISITOR_AS_HOME_AGAINST_3PA", "LAST_3_VISITOR_AS_HOME_AGAINST_3P%", "LAST_3_VISITOR_AS_HOME_AGAINST_FT", "LAST_3_VISITOR_AS_HOME_AGAINST_FTA", "LAST_3_VISITOR_AS_HOME_AGAINST_FT%", "LAST_3_VISITOR_AS_HOME_AGAINST_ORB", "LAST_3_VISITOR_AS_HOME_AGAINST_DRB", "LAST_3_VISITOR_AS_HOME_AGAINST_TRB", "LAST_3_VISITOR_AS_HOME_AGAINST_AST", "LAST_3_VISITOR_AS_HOME_AGAINST_STL", "LAST_3_VISITOR_AS_HOME_AGAINST_BLK", "LAST_3_VISITOR_AS_HOME_AGAINST_TOV", "LAST_3_VISITOR_AS_HOME_AGAINST_PF", "LAST_3_VISITOR_AS_HOME_AGAINST_PTS", "LAST_3_VISITOR_AS_HOME_AGAINST_TS%", "LAST_3_VISITOR_AS_HOME_AGAINST_eFG%", "LAST_3_VISITOR_AS_HOME_AGAINST_3PAr", "LAST_3_VISITOR_AS_HOME_AGAINST_FTr", "LAST_3_VISITOR_AS_HOME_AGAINST_ORB%", "LAST_3_VISITOR_AS_HOME_AGAINST_DRB%", "LAST_3_VISITOR_AS_HOME_AGAINST_TRB%", "LAST_3_VISITOR_AS_HOME_AGAINST_AST%", "LAST_3_VISITOR_AS_HOME_AGAINST_STL%", "LAST_3_VISITOR_AS_HOME_AGAINST_BLK%", "LAST_3_VISITOR_AS_HOME_AGAINST_TOV%", "LAST_3_VISITOR_AS_HOME_AGAINST_USG%", "LAST_3_VISITOR_AS_HOME_AGAINST_ORtg", "LAST_3_VISITOR_AS_HOME_AGAINST_DRtg",

"LAST_3_HOME_AS_HOME_FG", "LAST_3_HOME_AS_HOME_FGA", "LAST_3_HOME_AS_HOME_FG%", "LAST_3_HOME_AS_HOME_3P", "LAST_3_HOME_AS_HOME_3PA", "LAST_3_HOME_AS_HOME_3P%", "LAST_3_HOME_AS_HOME_FT", "LAST_3_HOME_AS_HOME_FTA", "LAST_3_HOME_AS_HOME_FT%", "LAST_3_HOME_AS_HOME_ORB", "LAST_3_HOME_AS_HOME_DRB", "LAST_3_HOME_AS_HOME_TRB", "LAST_3_HOME_AS_HOME_AST", "LAST_3_HOME_AS_HOME_STL", "LAST_3_HOME_AS_HOME_BLK", "LAST_3_HOME_AS_HOME_TOV", "LAST_3_HOME_AS_HOME_PF", "LAST_3_HOME_AS_HOME_PTS", "LAST_3_HOME_AS_HOME_TS%", "LAST_3_HOME_AS_HOME_eFG%", "LAST_3_HOME_AS_HOME_3PAr", "LAST_3_HOME_AS_HOME_FTr", "LAST_3_HOME_AS_HOME_ORB%", "LAST_3_HOME_AS_HOME_DRB%", "LAST_3_HOME_AS_HOME_TRB%", "LAST_3_HOME_AS_HOME_AST%", "LAST_3_HOME_AS_HOME_STL%", "LAST_3_HOME_AS_HOME_BLK%", "LAST_3_HOME_AS_HOME_TOV%", "LAST_3_HOME_AS_HOME_USG%", "LAST_3_HOME_AS_HOME_ORtg", "LAST_3_HOME_AS_HOME_DRtg",

"LAST_3_HOME_AS_HOME_AGAINST_FG", "LAST_3_HOME_AS_HOME_AGAINST_FGA", "LAST_3_HOME_AS_HOME_AGAINST_FG%", "LAST_3_HOME_AS_HOME_AGAINST_3P", "LAST_3_HOME_AS_HOME_AGAINST_3PA", "LAST_3_HOME_AS_HOME_AGAINST_3P%", "LAST_3_HOME_AS_HOME_AGAINST_FT", "LAST_3_HOME_AS_HOME_AGAINST_FTA", "LAST_3_HOME_AS_HOME_AGAINST_FT%", "LAST_3_HOME_AS_HOME_AGAINST_ORB", "LAST_3_HOME_AS_HOME_AGAINST_DRB", "LAST_3_HOME_AS_HOME_AGAINST_TRB", "LAST_3_HOME_AS_HOME_AGAINST_AST", "LAST_3_HOME_AS_HOME_AGAINST_STL", "LAST_3_HOME_AS_HOME_AGAINST_BLK", "LAST_3_HOME_AS_HOME_AGAINST_TOV", "LAST_3_HOME_AS_HOME_AGAINST_PF", "LAST_3_HOME_AS_HOME_AGAINST_PTS", "LAST_3_HOME_AS_HOME_AGAINST_TS%", "LAST_3_HOME_AS_HOME_AGAINST_eFG%", "LAST_3_HOME_AS_HOME_AGAINST_3PAr", "LAST_3_HOME_AS_HOME_AGAINST_FTr", "LAST_3_HOME_AS_HOME_AGAINST_ORB%", "LAST_3_HOME_AS_HOME_AGAINST_DRB%", "LAST_3_HOME_AS_HOME_AGAINST_TRB%", "LAST_3_HOME_AS_HOME_AGAINST_AST%", "LAST_3_HOME_AS_HOME_AGAINST_STL%", "LAST_3_HOME_AS_HOME_AGAINST_BLK%", "LAST_3_HOME_AS_HOME_AGAINST_TOV%", "LAST_3_HOME_AS_HOME_AGAINST_USG%", "LAST_3_HOME_AS_HOME_AGAINST_ORtg", "LAST_3_HOME_AS_HOME_AGAINST_DRtg",

"LAST_3_HOME_AS_VISITOR_FG", "LAST_3_HOME_AS_VISITOR_FGA", "LAST_3_HOME_AS_VISITOR_FG%", "LAST_3_HOME_AS_VISITOR_3P", "LAST_3_HOME_AS_VISITOR_3PA", "LAST_3_HOME_AS_VISITOR_3P%", "LAST_3_HOME_AS_VISITOR_FT", "LAST_3_HOME_AS_VISITOR_FTA", "LAST_3_HOME_AS_VISITOR_FT%", "LAST_3_HOME_AS_VISITOR_ORB", "LAST_3_HOME_AS_VISITOR_DRB", "LAST_3_HOME_AS_VISITOR_TRB", "LAST_3_HOME_AS_HOME_AST", "LAST_3_HOME_AS_VISITOR_STL", "LAST_3_HOME_AS_VISITOR_BLK", "LAST_3_HOME_AS_VISITOR_TOV", "LAST_3_HOME_AS_VISITOR_PF", "LAST_3_HOME_AS_VISITOR_PTS", "LAST_3_HOME_AS_VISITOR_TS%", "LAST_3_HOME_AS_VISITOR_eFG%", "LAST_3_HOME_AS_VISITOR_3PAr", "LAST_3_HOME_AS_VISITOR_FTr", "LAST_3_HOME_AS_VISITOR_ORB%", "LAST_3_HOME_AS_VISITOR_DRB%", "LAST_3_HOME_AS_VISITOR_TRB%", "LAST_3_HOME_AS_HOME_AST%", "LAST_3_HOME_AS_VISITOR_STL%", "LAST_3_HOME_AS_VISITOR_BLK%", "LAST_3_HOME_AS_VISITOR_TOV%", "LAST_3_HOME_AS_VISITOR_USG%", "LAST_3_HOME_AS_VISITOR_ORtg", "LAST_3_HOME_AS_VISITOR_DRtg",

"LAST_3_HOME_AS_VISITOR_AGAINST_FG", "LAST_3_HOME_AS_VISITOR_AGAINST_FGA", "LAST_3_HOME_AS_VISITOR_AGAINST_FG%", "LAST_3_HOME_AS_VISITOR_AGAINST_3P", "LAST_3_HOME_AS_VISITOR_AGAINST_3PA", "LAST_3_HOME_AS_VISITOR_AGAINST_3P%", "LAST_3_HOME_AS_VISITOR_AGAINST_FT", "LAST_3_HOME_AS_VISITOR_AGAINST_FTA", "LAST_3_HOME_AS_VISITOR_AGAINST_FT%", "LAST_3_HOME_AS_VISITOR_AGAINST_ORB", "LAST_3_HOME_AS_VISITOR_AGAINST_DRB", "LAST_3_HOME_AS_VISITOR_AGAINST_TRB", "LAST_3_HOME_AS_VISITOR_AGAINST_AST", "LAST_3_HOME_AS_VISITOR_AGAINST_STL", "LAST_3_HOME_AS_VISITOR_AGAINST_BLK", "LAST_3_HOME_AS_VISITOR_AGAINST_TOV", "LAST_3_HOME_AS_VISITOR_AGAINST_PF", "LAST_3_HOME_AS_VISITOR_AGAINST_PTS", "LAST_3_HOME_AS_VISITOR_AGAINST_TS%", "LAST_3_HOME_AS_VISITOR_AGAINST_eFG%", "LAST_3_HOME_AS_VISITOR_AGAINST_3PAr", "LAST_3_HOME_AS_VISITOR_AGAINST_FTr", "LAST_3_HOME_AS_VISITOR_AGAINST_ORB%", "LAST_3_HOME_AS_VISITOR_AGAINST_DRB%", "LAST_3_HOME_AS_VISITOR_AGAINST_TRB%", "LAST_3_HOME_AS_VISITOR_AGAINST_AST%", "LAST_3_HOME_AS_VISITOR_AGAINST_STL%", "LAST_3_HOME_AS_VISITOR_AGAINST_BLK%", "LAST_3_HOME_AS_VISITOR_AGAINST_TOV%", "LAST_3_HOME_AS_VISITOR_AGAINST_USG%", "LAST_3_HOME_AS_VISITOR_AGAINST_ORtg", "LAST_3_HOME_AS_VISITOR_AGAINST_DRtg",

"TOTAL_POINTS"]

newData= pd.DataFrame(pd.DataFrame(new_rows, columns=columns))

In [37]:
#Fill NA
newData = newData.fillna(-1)
newData.head()

,VISITOR_TEAM,HOME_TEAM,VISITOR_AS_VISITOR_FG,VISITOR_AS_VISITOR_FGA,VISITOR_AS_VISITOR_FG%,VISITOR_AS_VISITOR_3P,VISITOR_AS_VISITOR_3PA,VISITOR_AS_VISITOR_3P%,VISITOR_AS_VISITOR_FT,VISITOR_AS_VISITOR_FTA,...,LAST_3_HOME_AS_VISITOR_AGAINST_DRB%,LAST_3_HOME_AS_VISITOR_AGAINST_TRB%,LAST_3_HOME_AS_VISITOR_AGAINST_AST%,LAST_3_HOME_AS_VISITOR_AGAINST_STL%,LAST_3_HOME_AS_VISITOR_AGAINST_BLK%,LAST_3_HOME_AS_VISITOR_AGAINST_TOV%,LAST_3_HOME_AS_VISITOR_AGAINST_USG%,LAST_3_HOME_AS_VISITOR_AGAINST_ORtg,LAST_3_HOME_AS_VISITOR_AGAINST_DRtg,TOTAL_POINTS
0,1179,1192,35.555556,81.111111,0.438556,3.111111,12.777778,0.247333,17.888889,24.666667,...,71.800000,47.700000,55.633333,6.700000,4.066667,11.600000,100.0,108.366667,111.366667,178
1,1191,1146,34.500000,80.500000,0.429200,7.600000,18.900000,0.402400,19.700000,26.000000,...,76.666667,52.766667,59.400000,6.933333,10.200000,14.266667,100.0,108.766667,103.466667,205
2,1169,1197,31.285714,78.857143,0.396571,3.571429,15.000000,0.236429,21.285714,27.714286,...,70.433333,49.500000,56.733333,6.033333,6.566667,10.666667,100.0,114.600000,103.233333,191
3,1186,1183,37.875000,83.500000,0.453125,4.625000,17.500000,0.270625,17.125000,21.750000,...,76.666667,52.200000,50.500000,5.166667,5.833333,11.500000,100.0,110.100000,116.400000,202
4,1144,1156,35.666667,76.666667,0.465667,6.333333,19.666667,0.320333,25.000000,33.666667,...,71.566667,53.500000,63.766667,10.033333,6.200000,13.233333,100.0,116.600000,112.833333,208


In [38]:
#Store data with features
newData.to_csv('/Users/ruben.arana/git/beat-bookies/data/dataWithFeatures2006.csv', index=False)

In [39]:
newData.head()

,VISITOR_TEAM,HOME_TEAM,VISITOR_AS_VISITOR_FG,VISITOR_AS_VISITOR_FGA,VISITOR_AS_VISITOR_FG%,VISITOR_AS_VISITOR_3P,VISITOR_AS_VISITOR_3PA,VISITOR_AS_VISITOR_3P%,VISITOR_AS_VISITOR_FT,VISITOR_AS_VISITOR_FTA,...,LAST_3_HOME_AS_VISITOR_AGAINST_DRB%,LAST_3_HOME_AS_VISITOR_AGAINST_TRB%,LAST_3_HOME_AS_VISITOR_AGAINST_AST%,LAST_3_HOME_AS_VISITOR_AGAINST_STL%,LAST_3_HOME_AS_VISITOR_AGAINST_BLK%,LAST_3_HOME_AS_VISITOR_AGAINST_TOV%,LAST_3_HOME_AS_VISITOR_AGAINST_USG%,LAST_3_HOME_AS_VISITOR_AGAINST_ORtg,LAST_3_HOME_AS_VISITOR_AGAINST_DRtg,TOTAL_POINTS
0,1179,1192,35.555556,81.111111,0.438556,3.111111,12.777778,0.247333,17.888889,24.666667,...,71.800000,47.700000,55.633333,6.700000,4.066667,11.600000,100.0,108.366667,111.366667,178
1,1191,1146,34.500000,80.500000,0.429200,7.600000,18.900000,0.402400,19.700000,26.000000,...,76.666667,52.766667,59.400000,6.933333,10.200000,14.266667,100.0,108.766667,103.466667,205
2,1169,1197,31.285714,78.857143,0.396571,3.571429,15.000000,0.236429,21.285714,27.714286,...,70.433333,49.500000,56.733333,6.033333,6.566667,10.666667,100.0,114.600000,103.233333,191
3,1186,1183,37.875000,83.500000,0.453125,4.625000,17.500000,0.270625,17.125000,21.750000,...,76.666667,52.200000,50.500000,5.166667,5.833333,11.500000,100.0,110.100000,116.400000,202
4,1144,1156,35.666667,76.666667,0.465667,6.333333,19.666667,0.320333,25.000000,33.666667,...,71.566667,53.500000,63.766667,10.033333,6.200000,13.233333,100.0,116.600000,112.833333,208
